In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import pprint
pp = pprint.PrettyPrinter(indent=4)
%matplotlib inline

In [ ]:
DATASET_DIR = '../dataset/'

Merge Strategy

The label table is Evolução.  Evolução has two Id columns ('evolucao_Nº. Paciente' and 'evolucao_Nº da Intervenção')

We will merge in this order:

 evolucao << paciente (on 'evolucao_Nº. Paciente') << procedimento (on 'evolucao_Nº. Paciente' and 'evolucao_Nº da Intervenção') << complicacao (on 'evolucao_Nº. Paciente' and 'evolucao_Nº da Intervenção')


# Cleaning

#### Evolucao

In [ ]:
evolucao_df = pd.read_csv(DATASET_DIR+'evolucao.csv')
evolucao_df.drop(columns=['Unnamed: 0'], inplace=True)
evolucao_df

,evolucao_Nº. Paciente,evolucao_Nº da Intervenção,evolucao_Convênio,evolucao_Sucesso Clínico,evolucao_Infarto após Intervenção,evolucao_Tipo Infarto após Intervenção,evolucao_Óbito Hospitalar,evolucao_Causa do Óbito Hospitalar,evolucao_Oclusão,evolucao_Tratamento da Oclusão,evolucao_Complicação Vascular,evolucao_Tipo Complicação Vascular,evolucao_I.R.A,evolucao_I.R.A Dialítica,evolucao_A.V.C Hemorrágico,evolucao_A.V.C Isquêmico,evolucao_Sangramento TIMI,evolucao_Sangramento GUSTO,evolucao_Quantidade Vaso Tratado
0,5700,42353,S.U.S,Sim,Não,NaN,Não,NaN,NaN,NaN,Não,NaN,Não,Não,Não,Não,NaN,NaN,1
1,8300,46156,BRADESCO SAÚDE,Sim,Não,NaN,Não,NaN,NaN,NaN,Não,NaN,Não,Não,Não,Não,NaN,NaN,1
2,8500,45437,S.U.S,Sim,Não,NaN,Não,NaN,NaN,NaN,Não,NaN,Não,Não,Não,Não,NaN,NaN,2
3,1057,38316,S.U.S,Sim,Não,NaN,Não,NaN,NaN,NaN,Não,NaN,Não,Não,Não,Não,NaN,NaN,1
4,1061,57118,UNIMED,Sim,Não,NaN,Não,NaN,NaN,NaN,Não,NaN,Não,Não,Não,Não,NaN,NaN,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9634,50643,5421,FUND.CESP.,Sim,Não,NaN,Não,NaN,NaN,NaN,Não,NaN,Não,Não,Não,Não,NaN,NaN,3
9635,51552,5548,PARTICULAR,Não,Não,NaN,Sim,Cardíaca,NaN,NaN,Não,NaN,Não,Não,Não,Não,NaN,NaN,1
9636,51585,69452,AMIL,Sim,Não,NaN,Não,NaN,NaN,NaN,Não,NaN,Não,Não,Não,Não,NaN,NaN,2
9637,52140,5625,VERA CRUZ,Sim,Não,NaN,Não,NaN,NaN,NaN,Não,NaN,Não,Não,Não,Não,NaN,NaN,1


In [ ]:
# Select all duplicate rows based on one column
duplicateRowsDF = evolucao_df[evolucao_df.duplicated(['evolucao_Nº da Intervenção'], keep=False)]
duplicateRowsDF = duplicateRowsDF.sort_values(by=['evolucao_Nº da Intervenção'])
duplicateRowsDF

,evolucao_Nº. Paciente,evolucao_Nº da Intervenção,evolucao_Convênio,evolucao_Sucesso Clínico,evolucao_Infarto após Intervenção,evolucao_Tipo Infarto após Intervenção,evolucao_Óbito Hospitalar,evolucao_Causa do Óbito Hospitalar,evolucao_Oclusão,evolucao_Tratamento da Oclusão,evolucao_Complicação Vascular,evolucao_Tipo Complicação Vascular,evolucao_I.R.A,evolucao_I.R.A Dialítica,evolucao_A.V.C Hemorrágico,evolucao_A.V.C Isquêmico,evolucao_Sangramento TIMI,evolucao_Sangramento GUSTO,evolucao_Quantidade Vaso Tratado
19,1560,4,MEDIAL SAÚDE,Sim,Não,NaN,Não,NaN,NaN,NaN,Não,NaN,Não,Não,Não,Não,NaN,NaN,1
2773,12330,4,UNIMED,Sim,Não,NaN,Não,NaN,NaN,NaN,Não,NaN,Não,Não,Não,Não,NaN,NaN,1
8258,41017,4,NUCLEN,Sim,Não,NaN,Não,NaN,NaN,NaN,Não,NaN,Não,Não,Não,Não,NaN,NaN,1
8392,1619,9,FUND.CESP.,Sim,Não,NaN,Não,NaN,NaN,NaN,Não,NaN,Não,Não,Não,Não,NaN,NaN,1
7051,41021,9,PLAMER,Sim,Não,NaN,Não,NaN,NaN,NaN,Não,NaN,Não,Não,Não,Não,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
971,33573,62362,S.U.S,Sim,Não,NaN,Não,NaN,NaN,NaN,Não,NaN,Não,Não,Não,Não,NaN,NaN,2
9532,40458,66542,S.U.S,Sim,Não,NaN,Não,NaN,NaN,NaN,Não,NaN,Não,Não,Não,Não,NaN,NaN,1
1091,40533,66542,ECONOMUS,Sim,Não,NaN,Não,NaN,NaN,NaN,Não,NaN,Não,Não,Não,Não,NaN,NaN,2
2256,40620,66756,S.U.S,Sim,Não,NaN,Não,NaN,NaN,NaN,Não,NaN,Não,Não,Não,Não,NaN,NaN,1


In [ ]:
duplicatedCompositeKeys = (evolucao_df[evolucao_df.duplicated(subset=['evolucao_Nº. Paciente','evolucao_Nº da Intervenção'], keep=False)]).sort_values(by=['evolucao_Nº. Paciente'])
duplicatedCompositeKeys

,evolucao_Nº. Paciente,evolucao_Nº da Intervenção,evolucao_Convênio,evolucao_Sucesso Clínico,evolucao_Infarto após Intervenção,evolucao_Tipo Infarto após Intervenção,evolucao_Óbito Hospitalar,evolucao_Causa do Óbito Hospitalar,evolucao_Oclusão,evolucao_Tratamento da Oclusão,evolucao_Complicação Vascular,evolucao_Tipo Complicação Vascular,evolucao_I.R.A,evolucao_I.R.A Dialítica,evolucao_A.V.C Hemorrágico,evolucao_A.V.C Isquêmico,evolucao_Sangramento TIMI,evolucao_Sangramento GUSTO,evolucao_Quantidade Vaso Tratado
526,16145,51304,FUND.CESP.,Sim,Não,NaN,Não,NaN,NaN,NaN,Não,NaN,Não,Não,Não,Não,NaN,NaN,1
8921,16145,51304,CESP,Sim,Não,NaN,Não,NaN,NaN,NaN,Não,NaN,Não,Não,Não,Não,NaN,NaN,1
1862,19542,53590,S.U.S,Sim,Não,NaN,Não,NaN,NaN,NaN,Não,NaN,Não,Não,Não,Não,NaN,NaN,2
9049,19542,53590,S.U.S,Sim,Não,NaN,Não,NaN,NaN,NaN,Não,NaN,Não,Não,Não,Não,NaN,NaN,1


In [ ]:
evolucao_df[evolucao_df["evolucao_Nº. Paciente"] == 19542]

,evolucao_Nº. Paciente,evolucao_Nº da Intervenção,evolucao_Convênio,evolucao_Sucesso Clínico,evolucao_Infarto após Intervenção,evolucao_Tipo Infarto após Intervenção,evolucao_Óbito Hospitalar,evolucao_Causa do Óbito Hospitalar,evolucao_Oclusão,evolucao_Tratamento da Oclusão,evolucao_Complicação Vascular,evolucao_Tipo Complicação Vascular,evolucao_I.R.A,evolucao_I.R.A Dialítica,evolucao_A.V.C Hemorrágico,evolucao_A.V.C Isquêmico,evolucao_Sangramento TIMI,evolucao_Sangramento GUSTO,evolucao_Quantidade Vaso Tratado
1862,19542,53590,S.U.S,Sim,Não,NaN,Não,NaN,NaN,NaN,Não,NaN,Não,Não,Não,Não,NaN,NaN,2
4187,19542,53422,S.U.S,Sim,Não,NaN,Não,NaN,NaN,NaN,Não,NaN,Não,Não,Não,Não,NaN,NaN,1
9049,19542,53590,S.U.S,Sim,Não,NaN,Não,NaN,NaN,NaN,Não,NaN,Não,Não,Não,Não,NaN,NaN,1


We will have to drop patient 19542

#### Paciente

In [ ]:
paciente_df = pd.read_csv(DATASET_DIR+'paciente.csv')
paciente_df.drop(columns=['Unnamed: 0'], inplace=True)
paciente_df

,paciente_Nº. Paciente,paciente_Sexo,paciente_Idade Atual,paciente_Idade no procedimento,paciente_Raça,paciente_Procedência,paciente_Grau de Instrução,paciente_Faixa Etária,paciente_HAS,paciente_Diabetes,...,paciente_Pressão Arterial Diastólica,paciente_Frequência Respiratória,paciente_Temperatura (ºC),paciente_Cintura Abdominal (cm),paciente_Peso (Kg),paciente_Altura (metros),paciente_Índice de Massa Corporal,paciente_Creatinina Sérica,"paciente_MDRD (ml/min/1,73m²)",paciente_Clearance Cr (ml/min)
0,8200,Feminino,75,65,Branco,Internado,1ª a 4ª série,60-74,Não,Não,...,60-89,20.0,36.2,NaN,58.0,1.60,22.66,0.70,89.0,73.36
1,1017,Masculino,70,59,Branco,Ambulatorial,2º grau,50-59,Não,Não,...,60-89,16.0,NaN,NaN,61.0,1.70,21.11,NaN,NaN,NaN
2,1070,Masculino,56,44,Branco,Internado,5ª a 8ª série,40-49,Sim,Não,...,90-110,18.0,NaN,NaN,94.0,1.73,31.41,NaN,NaN,NaN
3,1090,Feminino,66,55,Branco,Ambulatorial,Não sabe/Sem declaração,50-59,Sim,Sim,...,NaN,NaN,NaN,NaN,86.0,1.57,34.89,NaN,NaN,NaN
4,1095,Feminino,90,79,Branco,Ambulatorial,Não sabe/Sem declaração,75-79,Sim,Sim,...,90-110,NaN,NaN,NaN,69.0,1.65,25.34,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8744,51208,Masculino,67,67,Branco,Ambulatorial,Não sabe/Sem declaração,60-74,Não,Sim,...,90-110,20.0,NaN,NaN,81.0,1.72,27.38,0.72,116.0,114.06
8745,51508,Masculino,56,56,Branco,Internado,2º grau,50-59,Sim,Sim,...,NaN,NaN,NaN,NaN,80.0,1.80,24.69,NaN,NaN,NaN
8746,51624,Masculino,55,54,Branco,Ambulatorial,Não sabe/Sem declaração,50-59,Sim,Não,...,60-89,20.0,36.5,NaN,86.0,1.80,26.54,1.20,67.0,85.60
8747,51914,Masculino,64,63,Branco,Internado,5ª a 8ª série,60-74,Não,Não,...,NaN,NaN,NaN,NaN,78.0,1.75,25.47,NaN,NaN,NaN


In [ ]:
# Select all duplicate rows based on one column
duplicateRowsDF = paciente_df[paciente_df.duplicated(['paciente_Nº. Paciente'], keep=False)]
duplicateRowsDF = duplicateRowsDF.sort_values(by=['paciente_Nº. Paciente'])
duplicateRowsDF

,paciente_Nº. Paciente,paciente_Sexo,paciente_Idade Atual,paciente_Idade no procedimento,paciente_Raça,paciente_Procedência,paciente_Grau de Instrução,paciente_Faixa Etária,paciente_HAS,paciente_Diabetes,...,paciente_Pressão Arterial Diastólica,paciente_Frequência Respiratória,paciente_Temperatura (ºC),paciente_Cintura Abdominal (cm),paciente_Peso (Kg),paciente_Altura (metros),paciente_Índice de Massa Corporal,paciente_Creatinina Sérica,"paciente_MDRD (ml/min/1,73m²)",paciente_Clearance Cr (ml/min)


#### Procedimento

In [ ]:
procedimento_df = pd.read_csv(DATASET_DIR+'procedimento.csv')
procedimento_df.drop(columns=['Unnamed: 0'], inplace=True)
procedimento_df

In [ ]:
# Select all duplicate rows based on one column
duplicateRowsDF = procedimento_df[procedimento_df.duplicated(['procedimento_Nº da Intervenção'], keep=False)]
duplicateRowsDF = duplicateRowsDF.sort_values(by=['procedimento_Nº da Intervenção'])
duplicateRowsDF

,procedimento_Nº. Paciente,procedimento_Nº da Intervenção,procedimento_Convênio,procedimento_Preceptor - Intervenção,procedimento_Operador 1 - Intervenção,procedimento_Operador 2 - Intervenção,procedimento_Anestesia - Intervenção,procedimento_Droga - Intervenção,procedimento_Técnica - Intervenção,procedimento_Introdutor - Intervenção,...,procedimento_Eptifibatide,procedimento_Adenosina,procedimento_Ultrasom Intracoronário,procedimento_Angiografia Quantitativa,procedimento_Inibidor de glicoproteína,procedimento_Clopidogrel.1,procedimento_Dose (mg),procedimento_Tempo origem/balão (minutos) - Primária,procedimento_Tempo dor/balão (minutos) - Primária,procedimento_Tempo Hosp. Band./balão (minutos) - Primária
2778,12330,4,UNIMED,e937c52858c5aa2da60074cc8c9a451ee6f7dbde,2de55d326a7ae5213c43d6f6e87df17951acbe66,NaN,Local,Xilocaína 2%,kissing-balloon,6F,...,Não,Não,Não,Sim,Nenhum,> 6h,300,NaN,NaN,NaN
7178,41017,4,NUCLEN,2de55d326a7ae5213c43d6f6e87df17951acbe66,2de55d326a7ae5213c43d6f6e87df17951acbe66,e937c52858c5aa2da60074cc8c9a451ee6f7dbde,Local,Xilocaína 2%,kissing-balloon,6F,...,Não,Não,Não,Não,Nenhum,< 6h,180,NaN,NaN,NaN
7295,1560,4,MEDIAL SAÚDE,b894e83fbaede3300547d89376a02e26e6720441,b894e83fbaede3300547d89376a02e26e6720441,NaN,NaN,NaN,NaN,6F,...,Não,Sim,Não,Sim,Agrastat,< 6h,300,NaN,NaN,NaN
6120,1619,9,FUND.CESP.,b894e83fbaede3300547d89376a02e26e6720441,b894e83fbaede3300547d89376a02e26e6720441,NaN,NaN,NaN,NaN,6F,...,Não,Não,Não,Sim,Nenhum,> 6h,300,NaN,NaN,NaN
2261,41021,9,PLAMER,2de55d326a7ae5213c43d6f6e87df17951acbe66,2de55d326a7ae5213c43d6f6e87df17951acbe66,NaN,Geral Sedação,Midazolan,kissing-balloon,5F,...,Não,Não,Não,Sim,Nenhum,> 6h,600,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4612,32868,62362,UNIMED,3b59fb20418b69a8a6dd2aab89d61e5a88445e51,773da479469dedb11b0f29fafdfb267ef3b6501f,NaN,Local,Xilocaína 2%,kissing-balloon,6F,...,Não,Não,Não,Sim,Nenhum,> 6h,75,NaN,NaN,NaN
4730,40458,66542,S.U.S,e937c52858c5aa2da60074cc8c9a451ee6f7dbde,e937c52858c5aa2da60074cc8c9a451ee6f7dbde,NaN,Local,Xilocaína 2%,kissing-balloon,6F,...,Não,Não,Não,Sim,Nenhum,> 6h,75,NaN,NaN,NaN
9535,40533,66542,ECONOMUS,619ee63819441a040c3ade7db6aed9f1a2f3e16f,619ee63819441a040c3ade7db6aed9f1a2f3e16f,NaN,Local,Xilocaína 2%,kissing-balloon,7F,...,Não,Não,Não,Sim,Nenhum,> 6h,300,NaN,NaN,NaN
7165,40620,66756,S.U.S,e937c52858c5aa2da60074cc8c9a451ee6f7dbde,e937c52858c5aa2da60074cc8c9a451ee6f7dbde,NaN,Local,Xilocaína 2%,kissing-balloon,6F,...,Não,Não,Não,Sim,Nenhum,> 6h,75,NaN,NaN,NaN


In [ ]:
duplicatedCompositeKeys = (procedimento_df[procedimento_df.duplicated(subset=['procedimento_Nº. Paciente','procedimento_Nº da Intervenção'], keep=False)]).sort_values(by=['procedimento_Nº. Paciente'])
duplicatedCompositeKeys

,procedimento_Nº. Paciente,procedimento_Nº da Intervenção,procedimento_Convênio,procedimento_Preceptor - Intervenção,procedimento_Operador 1 - Intervenção,procedimento_Operador 2 - Intervenção,procedimento_Anestesia - Intervenção,procedimento_Droga - Intervenção,procedimento_Técnica - Intervenção,procedimento_Introdutor - Intervenção,...,procedimento_Eptifibatide,procedimento_Adenosina,procedimento_Ultrasom Intracoronário,procedimento_Angiografia Quantitativa,procedimento_Inibidor de glicoproteína,procedimento_Clopidogrel.1,procedimento_Dose (mg),procedimento_Tempo origem/balão (minutos) - Primária,procedimento_Tempo dor/balão (minutos) - Primária,procedimento_Tempo Hosp. Band./balão (minutos) - Primária
15,1345,38508,S.U.S,e298ac91a0dce0151bbf909ad6fb9528f3686ffd,619ee63819441a040c3ade7db6aed9f1a2f3e16f,NaN,Local,Xilocaína 2%,Convencional,6F,...,Não,Não,Não,Sim,Nenhum,< 6h,300,NaN,NaN,NaN
1204,1345,38508,S.U.S,4ffa9ca2f69db5b2dcfcf298717bd738091369e9,619ee63819441a040c3ade7db6aed9f1a2f3e16f,NaN,Local,Xilocaína 2%,kissing-balloon,6F,...,Não,Não,Não,Sim,Nenhum,< 6h,300,NaN,NaN,NaN
134,5236,41857,MARÍTIMA,e298ac91a0dce0151bbf909ad6fb9528f3686ffd,619ee63819441a040c3ade7db6aed9f1a2f3e16f,NaN,Local,Xilocaína 2%,kissing-balloon,6F,...,Não,Não,Não,Sim,Nenhum,> 6h,300,NaN,NaN,NaN
8600,5236,41857,MARÍTIMA,e298ac91a0dce0151bbf909ad6fb9528f3686ffd,619ee63819441a040c3ade7db6aed9f1a2f3e16f,NaN,Local,Xilocaína 2%,kissing-balloon,6F,...,Não,Não,Não,Sim,Nenhum,> 6h,300,NaN,NaN,NaN
1575,12893,49176,S.U.S,6e995efee9727f885098efd8b55d6fa30bf5a693,fa403edee335d22a2adae70f313c54ab8658ee9b,NaN,Local,Xilocaína 2%,kissing-balloon,6F,...,Não,Não,Não,Sim,Reo-Pro,< 6h,600,180.0,240.0,20.0
5294,12893,49176,S.U.S,4ffa9ca2f69db5b2dcfcf298717bd738091369e9,cac9909afa49eebe6077d69833570b6c82920c9a,4ffa9ca2f69db5b2dcfcf298717bd738091369e9,Local,Xilocaína 2%,kissing-balloon,6F,...,Não,Não,Não,Sim,Reo-Pro,> 6h,600,NaN,NaN,NaN
5359,14672,50333,SULAMÉRICA,6e995efee9727f885098efd8b55d6fa30bf5a693,3b59fb20418b69a8a6dd2aab89d61e5a88445e51,f733014a0a2498e3a0b778d5ceab40915d45c205,Local,Xilocaína 2%,kissing-balloon,7F,...,Não,Sim,Não,Sim,Reo-Pro,> 6h,75,30.0,30.0,15.0
7747,14672,50333,SULAMÉRICA,9403d5e47bacc4c7f1253769f52547ce8c14a9da,5fdf7b129baf0e2a7f95e652792819d14685e8dd,f733014a0a2498e3a0b778d5ceab40915d45c205,Local,Xilocaína 2%,kissing-balloon,7F,...,Não,Não,Não,Sim,Nenhum,> 6h,75,NaN,NaN,NaN
1700,16145,51304,FUND.CESP.,6e995efee9727f885098efd8b55d6fa30bf5a693,5fdf7b129baf0e2a7f95e652792819d14685e8dd,6a6c10014811828bb15ad6b89b34ba834a5c14d6,Local,Xilocaína 2%,kissing-balloon,7F,...,Não,Não,Não,Sim,Nenhum,> 6h,75,NaN,NaN,NaN
7814,16145,51304,CESP,6e995efee9727f885098efd8b55d6fa30bf5a693,5fdf7b129baf0e2a7f95e652792819d14685e8dd,03d49dfd083cedb05d8d211d4202f91f8ddb58f4,Local,Xilocaína 2%,kissing-balloon,5F,...,Não,Não,Não,Sim,Nenhum,> 6h,75,NaN,NaN,NaN


In [ ]:
duplicatedCompositeKeys.index

Int64Index([15, 1204, 134, 8600, 1575, 5294, 5359, 7747, 1700, 7814, 1706,
            2938, 664, 7923],
           dtype='int64')

Esses valores nos fizeram perceber que existe mais d eum paciente por ID de intervenção, logo a chave precisa ser composta
Essas combinações apresentam mais de uma combinação por paciente e intervenção na tabela de procedimentos e serão excluidos para evitar inconsistencias, dado que não sabemos o motivo

In [ ]:
procedimento_df.drop(index=duplicatedCompositeKeys.index, inplace=True, errors='raise')

In [ ]:
procedimento_df

,procedimento_Nº. Paciente,procedimento_Nº da Intervenção,procedimento_Convênio,procedimento_Preceptor - Intervenção,procedimento_Operador 1 - Intervenção,procedimento_Operador 2 - Intervenção,procedimento_Anestesia - Intervenção,procedimento_Droga - Intervenção,procedimento_Técnica - Intervenção,procedimento_Introdutor - Intervenção,...,procedimento_Eptifibatide,procedimento_Adenosina,procedimento_Ultrasom Intracoronário,procedimento_Angiografia Quantitativa,procedimento_Inibidor de glicoproteína,procedimento_Clopidogrel.1,procedimento_Dose (mg),procedimento_Tempo origem/balão (minutos) - Primária,procedimento_Tempo dor/balão (minutos) - Primária,procedimento_Tempo Hosp. Band./balão (minutos) - Primária
0,8500,45437,S.U.S,b894e83fbaede3300547d89376a02e26e6720441,b894e83fbaede3300547d89376a02e26e6720441,619ee63819441a040c3ade7db6aed9f1a2f3e16f,Local,Xilocaína 2%,kissing-balloon,7F,...,Não,Não,Não,Sim,Nenhum,> 6h,600,NaN,NaN,NaN
1,8600,45547,SÃO CRISTÓVÃO,9403d5e47bacc4c7f1253769f52547ce8c14a9da,fa403edee335d22a2adae70f313c54ab8658ee9b,5b00c14bee173006a6405bf69ce7ca970f120ef1,Local,Xilocaína 2%,kissing-balloon,6F,...,Não,Não,Não,Sim,Nenhum,< 6h,600,NaN,NaN,NaN
2,1073,38184,S.U.S,e937c52858c5aa2da60074cc8c9a451ee6f7dbde,619ee63819441a040c3ade7db6aed9f1a2f3e16f,NaN,Local,Xilocaína 2%,kissing-balloon,6F,...,Não,Não,Não,Sim,Nenhum,> 6h,300,NaN,NaN,NaN
3,1079,38192,S.U.S,2de55d326a7ae5213c43d6f6e87df17951acbe66,b5347fa43dc0b84172e05bba58ed2abc2e4fbe8e,NaN,Local,Xilocaína 2%,kissing-balloon,6F,...,Não,Não,Não,Sim,Nenhum,< 6h,300,NaN,NaN,NaN
4,1126,38267,UNIMED,e7cf00e9e59d52ebe23d0e7d39b7c645a630c387,619ee63819441a040c3ade7db6aed9f1a2f3e16f,NaN,Local,Xilocaína 2%,kissing-balloon,6F,...,Não,Não,Não,Sim,Nenhum,> 6h,300,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9639,51208,5639,ECONOMUS,b894e83fbaede3300547d89376a02e26e6720441,b894e83fbaede3300547d89376a02e26e6720441,3bc773389003e9045cdcbca2f60d4b12462e4f3e,Local,Xilocaína 2%,kissing-balloon,6F,...,Não,Sim,Não,Sim,Nenhum,> 6h,75,NaN,NaN,NaN
9640,51552,5548,PARTICULAR,b894e83fbaede3300547d89376a02e26e6720441,b894e83fbaede3300547d89376a02e26e6720441,NaN,Geral Inalatória,Xilocaína 2%,kissing-balloon,7F,...,Não,Não,Não,Não,Agrastat,< 6h,180,73.0,105.0,20.0
9641,51908,5595,SULAMÉRICA,5b00c14bee173006a6405bf69ce7ca970f120ef1,5b00c14bee173006a6405bf69ce7ca970f120ef1,NaN,Local,Xilocaína 2%,kissing-balloon,6F,...,Não,Sim,Não,Sim,Nenhum,> 6h,90,NaN,NaN,NaN
9642,52223,1454,SULAMÉRICA,6e995efee9727f885098efd8b55d6fa30bf5a693,6e995efee9727f885098efd8b55d6fa30bf5a693,NaN,Geral Sedação,Fentanil,kissing-balloon,6F,...,Não,Não,Não,Não,Reo-Pro,< 6h,0,1577.0,1680.0,54.0


#### Complicação

In [ ]:
complicacao_df = pd.read_csv(DATASET_DIR+'complicacao.csv')
complicacao_df.drop(columns=['Unnamed: 0'], inplace=True)
complicacao_df

,complicacao_Nº. Paciente,complicacao_Nº da Intervenção,complicacao_Procedimento,complicacao_Quando,complicacao_Destino Alta,complicacao_Contato Realizado,complicacao_Complicação,complicacao_Óbito,complicacao_Tipo Complicação,complicacao_Grau - Complicação,complicacao_Conduta
0,1006,38112,Terapêutico,Intra-exame até a alta,Domiciliar,NaN,Sim,Não,Vascular,Moderado,NaN
1,1639,547,Terapêutico,Intra-exame até a alta,Enfermaria,NaN,Sim,Não,Alérgica,Moderado,NaN
2,3406,40228,Terapêutico,Intra-exame até a alta,Domiciliar,NaN,Sim,Não,Hematoma,Leve,NaN
3,3809,40842,Terapêutico,Intra-exame até a alta,Enfermaria,NaN,Sim,Não,Isquêmica,Moderado,NaN
4,3863,40561,Terapêutico,Intra-exame até a alta,Enfermaria,NaN,Sim,Não,Isquêmica,Grave,NaN
...,...,...,...,...,...,...,...,...,...,...,...
1108,46889,68324,Terapêutico,Até 1 semana,NaN,NaN,Sim,Não,Isquêmica,Moderado,Retorno para avaliação Médica
1109,48805,5122,Terapêutico,Sala de exame,NaN,NaN,Sim,Não,Embólica,Leve,NaN
1110,49534,5219,Terapêutico,Sala de exame,NaN,NaN,Sim,Não,Isquêmica,Grave,NaN
1111,50715,1170,Terapêutico,Até 24 horas,NaN,NaN,Sim,Sim,Isquêmica,Grave,Orientação Telefônica


In [ ]:
# Select all duplicate rows based on one column
duplicateRowsDF = complicacao_df[complicacao_df.duplicated(['complicacao_Nº da Intervenção'], keep=False)]
duplicateRowsDF = duplicateRowsDF.sort_values(by=['complicacao_Nº da Intervenção'])
duplicateRowsDF

,complicacao_Nº. Paciente,complicacao_Nº da Intervenção,complicacao_Procedimento,complicacao_Quando,complicacao_Destino Alta,complicacao_Contato Realizado,complicacao_Complicação,complicacao_Óbito,complicacao_Tipo Complicação,complicacao_Grau - Complicação,complicacao_Conduta
992,8119,198,Terapêutico,Até 1 semana,NaN,NaN,Sim,Não,Hematoma,Moderado,Retorno para avaliação Médica
154,8119,198,Terapêutico,Até 1 semana,NaN,NaN,Sim,Não,Hematoma,Grave,Retorno para avaliação Médica
254,27228,261,Terapêutico,Sala de exame,NaN,NaN,Sim,Não,Vagal,Moderado,NaN
304,8954,261,Terapêutico,Até 1 semana,NaN,NaN,Sim,Não,Hematoma,Moderado,Orientação Telefônica
973,1639,547,Terapêutico,Intra-exame até a alta,Enfermaria,NaN,Sim,Não,Vagal,Moderado,NaN
...,...,...,...,...,...,...,...,...,...,...,...
1032,16217,62044,Terapêutico,Recuperação,NaN,NaN,Sim,Não,Hematoma,Moderado,NaN
962,39917,66116,Terapêutico,Recuperação,NaN,NaN,Sim,Não,Isquêmica,Grave,NaN
963,39917,66116,Terapêutico,Até 1 semana,NaN,NaN,Sim,Sim,Isquêmica,Grave,Retorno para avaliação Médica
1103,41572,66938,Terapêutico,Até 1 semana,NaN,NaN,Sim,Não,Hematoma,Leve,Orientação Telefônica


In [ ]:
duplicatedCompositeKeys = (complicacao_df[complicacao_df.duplicated(subset=['complicacao_Nº. Paciente','complicacao_Nº da Intervenção'], keep=False)]).sort_values(by=['complicacao_Nº. Paciente'])
duplicatedCompositeKeys

,complicacao_Nº. Paciente,complicacao_Nº da Intervenção,complicacao_Procedimento,complicacao_Quando,complicacao_Destino Alta,complicacao_Contato Realizado,complicacao_Complicação,complicacao_Óbito,complicacao_Tipo Complicação,complicacao_Grau - Complicação,complicacao_Conduta
0,1006,38112,Terapêutico,Intra-exame até a alta,Domiciliar,NaN,Sim,Não,Vascular,Moderado,NaN
286,1006,38112,Terapêutico,Até 1 semana,NaN,NaN,Sim,Não,Vascular,Leve,NaN
136,1142,38727,Terapêutico,Intra-exame até a alta,Enfermaria,NaN,Sim,Não,Isquêmica,Grave,NaN
702,1142,38727,Terapêutico,Intra-exame até a alta,Domiciliar,NaN,Sim,Não,Isquêmica,Grave,NaN
1,1639,547,Terapêutico,Intra-exame até a alta,Enfermaria,NaN,Sim,Não,Alérgica,Moderado,NaN
...,...,...,...,...,...,...,...,...,...,...,...
285,50715,1170,Terapêutico,Recuperação,NaN,NaN,Sim,Sim,Isquêmica,Grave,NaN
284,50715,1170,Terapêutico,Recuperação,NaN,NaN,Sim,Sim,Arritmia,Grave,NaN
1111,50715,1170,Terapêutico,Até 24 horas,NaN,NaN,Sim,Sim,Isquêmica,Grave,Orientação Telefônica
699,51479,5533,Terapêutico,Sala de exame,NaN,NaN,Sim,Não,Vascular,Grave,NaN


#### Balão

In [ ]:
balao_df = pd.read_csv(DATASET_DIR+'balao.csv')
balao_df.drop(columns=['Unnamed: 0'], inplace=True)
balao_df

,balao_Nº. Paciente,balao_Nº da Intervenção,balao_Convênio,balao_Diâmetro Cateter - Balão,balao_Comprimento Cateter - Balão,balao_Device Adjunto - Balão,balao_Pressão Final (ATM),balao_Resultado Angiográfico - Balão,balao_Tipo - Dissecção - Balão,balao_Grau de estenose pós,...,balao_Total Injetado (mg) - Monocordil - Balão,balao_Nitroprussiato - Balão,balao_N. Injeções - Nitroprussiato - Balão,balao_Dose/Injeção (mg) - Nitroprussiato - Balão,balao_Total Injetado (mg) - Nitroprussiato - Balão,balao_Adrenalina - Balão,balao_N. Injeções - Adrenalina - Balão,balao_Dose/Injeção (mg) - Adrenalina - Balão,balao_Total Injetado (mg) - Adrenalina - Balão,balao_Vaso coronário - Balão
0,1146,38274,SÃO CRISTÓVÃO,2.50,15.0,NaN,16.0,S - Sucesso,NaN,10.0,...,NaN,Não,NaN,NaN,NaN,Não,NaN,NaN,NaN,MamáriaDA
1,1394,493,VERA CRUZ,2.50,12.0,NaN,10.0,S - Sucesso,NaN,12.0,...,NaN,Não,NaN,NaN,NaN,Não,NaN,NaN,NaN,CX
2,1444,38618,S.U.S,3.50,15.0,NaN,18.0,S - Sucesso,NaN,10.0,...,NaN,Não,NaN,NaN,NaN,Não,NaN,NaN,NaN,DA
3,1470,38650,S.U.S,2.50,15.0,NaN,6.0,I2 - Ultrapassou e não dilatou,NaN,80.0,...,NaN,Não,NaN,NaN,NaN,Não,NaN,NaN,NaN,DA
4,1801,38340,S.U.S,2.00,15.0,NaN,14.0,S - Sucesso,NaN,40.0,...,NaN,Não,NaN,NaN,NaN,Não,NaN,NaN,NaN,Dg2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1170,42922,752,PARTICULAR,2.50,12.0,NaN,18.0,S - Sucesso,NaN,0.0,...,NaN,Não,NaN,NaN,NaN,Não,NaN,NaN,NaN,CD
1171,45344,938,AMIL,2.00,12.0,NaN,0.0,I1 - Não ultrapassou a lesão,NaN,90.0,...,NaN,Não,NaN,NaN,NaN,Não,NaN,NaN,NaN,CD
1172,45396,4637,SULAMÉRICA,1.00,9.0,NaN,0.0,S - Sucesso,NaN,0.0,...,NaN,Não,NaN,NaN,NaN,Não,NaN,NaN,NaN,Dg2
1173,48184,5019,CABESP,2.50,15.0,NaN,8.0,S - Sucesso,NaN,0.0,...,NaN,Não,NaN,NaN,NaN,Não,NaN,NaN,NaN,VPD


In [ ]:
# Select all duplicate rows based on one column
duplicateRowsDF = balao_df[balao_df.duplicated(['balao_Nº da Intervenção'], keep=False)]
duplicateRowsDF = duplicateRowsDF.sort_values(by=['balao_Nº da Intervenção'])
duplicateRowsDF

,balao_Nº. Paciente,balao_Nº da Intervenção,balao_Convênio,balao_Diâmetro Cateter - Balão,balao_Comprimento Cateter - Balão,balao_Device Adjunto - Balão,balao_Pressão Final (ATM),balao_Resultado Angiográfico - Balão,balao_Tipo - Dissecção - Balão,balao_Grau de estenose pós,...,balao_Total Injetado (mg) - Monocordil - Balão,balao_Nitroprussiato - Balão,balao_N. Injeções - Nitroprussiato - Balão,balao_Dose/Injeção (mg) - Nitroprussiato - Balão,balao_Total Injetado (mg) - Nitroprussiato - Balão,balao_Adrenalina - Balão,balao_N. Injeções - Adrenalina - Balão,balao_Dose/Injeção (mg) - Adrenalina - Balão,balao_Total Injetado (mg) - Adrenalina - Balão,balao_Vaso coronário - Balão
517,23771,187,UNIMED,2.50,12.0,NaN,0.0,I1 - Não ultrapassou a lesão,NaN,70.0,...,NaN,Não,NaN,NaN,NaN,Não,NaN,NaN,NaN,Mg1
722,5475,187,VERA CRUZ,2.75,18.0,NaN,8.0,S - Sucesso,NaN,15.0,...,NaN,Não,NaN,NaN,NaN,Não,NaN,NaN,NaN,DPD
817,23771,187,UNIMED,2.00,12.0,NaN,14.0,S - Sucesso,NaN,30.0,...,NaN,Não,NaN,NaN,NaN,Não,NaN,NaN,NaN,DA
298,7403,247,SPSM,2.50,9.0,NaN,10.0,I3 - Oclusão Aguda,NaN,100.0,...,NaN,Não,NaN,NaN,NaN,Sim,3.0,1.0,3.0,CD
30,7403,247,SPSM,1.50,15.0,NaN,10.0,I2 - Ultrapassou e não dilatou,NaN,100.0,...,NaN,Não,NaN,NaN,NaN,Não,NaN,NaN,NaN,CD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,33110,62050,S.U.S,3.50,20.0,NaN,14.0,S - Sucesso,NaN,10.0,...,NaN,Não,NaN,NaN,NaN,Não,NaN,NaN,NaN,CD
845,39048,65614,BRADESCO SAÚDE,2.00,9.0,NaN,10.0,S - Sucesso,NaN,0.0,...,NaN,Não,NaN,NaN,NaN,Não,NaN,NaN,NaN,DA
997,39048,65614,BRADESCO SAÚDE,2.00,20.0,NaN,12.0,S - Sucesso,NaN,0.0,...,NaN,Não,NaN,NaN,NaN,Não,NaN,NaN,NaN,Dg2
1159,39536,65899,BRADESCO SAÚDE,2.50,12.0,NaN,12.0,I1 - Não ultrapassou a lesão,NaN,100.0,...,NaN,Não,NaN,NaN,NaN,Não,NaN,NaN,NaN,DA


In [ ]:
duplicatedCompositeKeys = (balao_df[balao_df.duplicated(subset=['balao_Nº. Paciente','balao_Nº da Intervenção'], keep=False)]).sort_values(by=['balao_Nº. Paciente'])
duplicatedCompositeKeys

,balao_Nº. Paciente,balao_Nº da Intervenção,balao_Convênio,balao_Diâmetro Cateter - Balão,balao_Comprimento Cateter - Balão,balao_Device Adjunto - Balão,balao_Pressão Final (ATM),balao_Resultado Angiográfico - Balão,balao_Tipo - Dissecção - Balão,balao_Grau de estenose pós,...,balao_Total Injetado (mg) - Monocordil - Balão,balao_Nitroprussiato - Balão,balao_N. Injeções - Nitroprussiato - Balão,balao_Dose/Injeção (mg) - Nitroprussiato - Balão,balao_Total Injetado (mg) - Nitroprussiato - Balão,balao_Adrenalina - Balão,balao_N. Injeções - Adrenalina - Balão,balao_Dose/Injeção (mg) - Adrenalina - Balão,balao_Total Injetado (mg) - Adrenalina - Balão,balao_Vaso coronário - Balão
860,1429,38607,S.U.S,3.50,9.0,NaN,13.0,S - Sucesso,NaN,50.0,...,NaN,Não,NaN,NaN,NaN,Não,NaN,NaN,NaN,CX
861,1429,38607,S.U.S,3.50,15.0,NaN,16.0,S - Sucesso,NaN,3.0,...,NaN,Não,NaN,NaN,NaN,Não,NaN,NaN,NaN,CX
412,1518,507,VERA CRUZ,2.25,15.0,NaN,8.0,S - Sucesso,NaN,12.0,...,NaN,Não,NaN,NaN,NaN,Não,NaN,NaN,NaN,DPD
862,1518,507,VERA CRUZ,2.50,12.0,NaN,8.0,S - Sucesso,NaN,10.0,...,NaN,Não,NaN,NaN,NaN,Não,NaN,NaN,NaN,VPD
561,2260,39340,CRUZEIRO DO SUL,3.00,12.0,NaN,12.0,I1 - Não ultrapassou a lesão,NaN,70.0,...,NaN,Não,NaN,NaN,NaN,Não,NaN,NaN,NaN,CD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
406,48262,969,SF,1.50,12.0,NaN,20.0,I1 - Não ultrapassou a lesão,NaN,99.0,...,NaN,Não,NaN,NaN,NaN,Não,NaN,NaN,NaN,CD
1010,48262,969,SF,2.50,15.0,NaN,20.0,I1 - Não ultrapassou a lesão,NaN,99.0,...,NaN,Não,NaN,NaN,NaN,Não,NaN,NaN,NaN,CD
554,48262,969,SF,1.25,9.0,NaN,20.0,I1 - Não ultrapassou a lesão,NaN,99.0,...,NaN,Não,NaN,NaN,NaN,Não,NaN,NaN,NaN,CD
266,49050,5162,AMIL,1.25,9.0,NaN,0.0,I1 - Não ultrapassou a lesão,NaN,95.0,...,NaN,Não,NaN,NaN,NaN,Não,NaN,NaN,NaN,PVSDg1


#### Stent

In [ ]:
stent_df = pd.read_csv(DATASET_DIR+'stent.csv')
stent_df.drop(columns=['Unnamed: 0'], inplace=True)
stent_df

,stent_Nº. Paciente,stent_Nº da Intervenção,stent_Convênio,stent_Vaso coronário - Stent,stent_Stent,stent_Estratégia,stent_Diâmetro Dispositivo - Stent,stent_Comprimento Dispositivo - Stent,stent_Diâmetro Cateter Balão Adjunto Pré,stent_Diâmetro Cateter Balão Adjunto Pós,...,stent_Dose/Injeção (mg) - Monocordil - Stent,stent_Total Injetado (mg) - Monocordil - Stent,stent_Nitroprussiato - Stent,stent_N. Injeções - Nitroprussiato - Stent,stent_Dose/Injeção (mg) - Nitroprussiato - Stent,stent_Total Injetado (mg) - Nitroprussiato - Stent,stent_Adrenalina - Stent,stent_N. Injeções - Adrenalina - Stent,stent_Dose/Injeção (mg) - Adrenalina - Stent,stent_Total Injetado (mg) - Adrenalina - Stent
0,1061,38209,UNIMED,CX,LIBERTÉ,Stent Direto,3.00,8,NaN,NaN,...,NaN,NaN,Não,NaN,NaN,NaN,Não,NaN,NaN,NaN
1,1064,38175,S.U.S,DA,PRO-KINETIC,Pré-dilatação,3.00,18,2.5,NaN,...,NaN,NaN,Não,NaN,NaN,NaN,Não,NaN,NaN,NaN
2,1095,38208,S.U.S,Intermédio-RI,LIBERTÉ,Stent Direto,2.50,16,NaN,NaN,...,NaN,NaN,Não,NaN,NaN,NaN,Não,NaN,NaN,NaN
3,1144,38404,S.U.S,MamáriaDA,LIBERTÉ,Stent Direto,2.25,8,NaN,NaN,...,NaN,NaN,Não,NaN,NaN,NaN,Não,NaN,NaN,NaN
4,1167,61763,S.U.S,CD,LIBERTÉ,Pós-dilatação,4.00,20,NaN,4.5,...,NaN,NaN,Não,NaN,NaN,NaN,Não,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12374,51063,5474,VERA CRUZ,CD,ULTIMASTER,Stent Direto,3.50,15,NaN,NaN,...,NaN,NaN,Não,NaN,NaN,NaN,Não,NaN,NaN,NaN
12375,51088,5480,UNIMED,Dg1,XIENCE XPEDITION,Pré e Pós dilatação,2.50,23,2.0,2.5,...,NaN,NaN,Não,NaN,NaN,NaN,Não,NaN,NaN,NaN
12376,51479,5533,VERA CRUZ,CD,XIENCE XPEDITION,Pós-dilatação,3.50,8,NaN,3.5,...,NaN,NaN,Não,NaN,NaN,NaN,Não,NaN,NaN,NaN
12377,51508,1280,SF,DA,RESOLUTE INTEGRITY,Pré e Pós dilatação,2.75,14,2.5,3.5,...,NaN,NaN,Não,NaN,NaN,NaN,Não,NaN,NaN,NaN


In [ ]:
# Select all duplicate rows based on one column
duplicateRowsDF = stent_df[stent_df.duplicated(['stent_Nº da Intervenção'], keep=False)]
duplicateRowsDF = duplicateRowsDF.sort_values(by=['stent_Nº da Intervenção'])
duplicateRowsDF

,stent_Nº. Paciente,stent_Nº da Intervenção,stent_Convênio,stent_Vaso coronário - Stent,stent_Stent,stent_Estratégia,stent_Diâmetro Dispositivo - Stent,stent_Comprimento Dispositivo - Stent,stent_Diâmetro Cateter Balão Adjunto Pré,stent_Diâmetro Cateter Balão Adjunto Pós,...,stent_Dose/Injeção (mg) - Monocordil - Stent,stent_Total Injetado (mg) - Monocordil - Stent,stent_Nitroprussiato - Stent,stent_N. Injeções - Nitroprussiato - Stent,stent_Dose/Injeção (mg) - Nitroprussiato - Stent,stent_Total Injetado (mg) - Nitroprussiato - Stent,stent_Adrenalina - Stent,stent_N. Injeções - Adrenalina - Stent,stent_Dose/Injeção (mg) - Adrenalina - Stent,stent_Total Injetado (mg) - Adrenalina - Stent
9306,1560,4,MEDIAL SAÚDE,CD,LIBERTÉ,Stent Direto,3.00,16,NaN,NaN,...,NaN,NaN,Não,NaN,NaN,NaN,Não,NaN,NaN,NaN
1397,41017,4,NUCLEN,DA,RESOLUTE,Stent Direto,2.50,14,NaN,NaN,...,NaN,NaN,Não,NaN,NaN,NaN,Não,NaN,NaN,NaN
11316,12330,4,UNIMED,MamáriaCD,XIENCE V,Pré-dilatação,2.75,23,2.5,NaN,...,NaN,NaN,Não,NaN,NaN,NaN,Não,NaN,NaN,NaN
11329,12599,7,UNIMED,CX,LIBERTÉ,Stent Direto,4.00,12,NaN,NaN,...,NaN,NaN,Não,NaN,NaN,NaN,Não,NaN,NaN,NaN
9805,12599,7,UNIMED,DA,LIBERTÉ,Stent Direto,3.50,12,NaN,NaN,...,NaN,NaN,Não,NaN,NaN,NaN,Não,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12342,46285,69416,NOTRE DAME,CD,RESOLUTE INTEGRITY,Pré e Pós dilatação,4.00,30,3.0,4.0,...,NaN,NaN,Não,NaN,NaN,NaN,Não,NaN,NaN,NaN
6119,46285,69416,NOTRE DAME,VPE,RESOLUTE INTEGRITY,Pré e Pós dilatação,2.50,30,2.0,3.0,...,NaN,NaN,Não,NaN,NaN,NaN,Não,NaN,NaN,NaN
3030,46285,69416,NOTRE DAME,CD,RESOLUTE INTEGRITY,Pós-dilatação,3.50,26,NaN,4.0,...,NaN,NaN,Não,NaN,NaN,NaN,Não,NaN,NaN,NaN
6160,51585,69452,AMIL,Mg2,XIENCE V,Pré e Pós dilatação,2.50,38,2.5,3.0,...,NaN,NaN,Não,NaN,NaN,NaN,Não,NaN,NaN,NaN


In [ ]:
duplicatedCompositeKeys = (stent_df[stent_df.duplicated(subset=['stent_Nº. Paciente','stent_Nº da Intervenção'], keep=False)]).sort_values(by=['stent_Nº. Paciente'])
duplicatedCompositeKeys

,stent_Nº. Paciente,stent_Nº da Intervenção,stent_Convênio,stent_Vaso coronário - Stent,stent_Stent,stent_Estratégia,stent_Diâmetro Dispositivo - Stent,stent_Comprimento Dispositivo - Stent,stent_Diâmetro Cateter Balão Adjunto Pré,stent_Diâmetro Cateter Balão Adjunto Pós,...,stent_Dose/Injeção (mg) - Monocordil - Stent,stent_Total Injetado (mg) - Monocordil - Stent,stent_Nitroprussiato - Stent,stent_N. Injeções - Nitroprussiato - Stent,stent_Dose/Injeção (mg) - Nitroprussiato - Stent,stent_Total Injetado (mg) - Nitroprussiato - Stent,stent_Adrenalina - Stent,stent_N. Injeções - Adrenalina - Stent,stent_Dose/Injeção (mg) - Adrenalina - Stent,stent_Total Injetado (mg) - Adrenalina - Stent
10848,1057,38316,S.U.S,CD,LIBERTÉ,Stent Direto,3.00,16,NaN,NaN,...,NaN,NaN,Não,NaN,NaN,NaN,Não,NaN,NaN,NaN
10849,1057,38316,S.U.S,CD,LIBERTÉ,Stent Direto,3.00,12,NaN,NaN,...,NaN,NaN,Não,NaN,NaN,NaN,Não,NaN,NaN,NaN
6166,1061,57118,UNIMED,CX,XIENCE PRIME,Stent Direto,3.00,23,NaN,NaN,...,NaN,NaN,Não,NaN,NaN,NaN,Não,NaN,NaN,NaN
1542,1061,57118,UNIMED,DA,XIENCE PRIME,Pós-dilatação,2.50,15,NaN,2.5,...,NaN,NaN,Não,NaN,NaN,NaN,Não,NaN,NaN,NaN
1,1064,38175,S.U.S,DA,PRO-KINETIC,Pré-dilatação,3.00,18,2.50,NaN,...,NaN,NaN,Não,NaN,NaN,NaN,Não,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6162,51944,5635,VERA CRUZ,MamáriaDg,XIENCE XPEDITION,Pré e Pós dilatação,2.25,15,2.25,2.5,...,NaN,NaN,Não,NaN,NaN,NaN,Não,NaN,NaN,NaN
9269,51944,5635,VERA CRUZ,MamáriaDA,XIENCE XPEDITION,Pré e Pós dilatação,2.25,20,2.00,2.5,...,NaN,NaN,Não,NaN,NaN,NaN,Não,NaN,NaN,NaN
4630,52043,1296,PARTICULAR,DA,RESOLUTE INTEGRITY,Pré e Pós dilatação,3.00,18,2.25,3.5,...,NaN,NaN,Não,NaN,NaN,NaN,Não,NaN,NaN,NaN
3081,52043,1296,PARTICULAR,Mg2,RESOLUTE INTEGRITY,Pré-dilatação,2.25,14,2.00,NaN,...,NaN,NaN,Não,NaN,NaN,NaN,Não,NaN,NaN,NaN


#### Vaso

In [ ]:
vaso_df = pd.read_csv(DATASET_DIR+'vaso.csv')
vaso_df.drop(columns=['Unnamed: 0'], inplace=True)
vaso_df

,vaso_Nº. Paciente,vaso_Nº da Intervenção,vaso_Convênio,vaso_Anatomia - Intervenção,vaso_Vaso coronário - Intervenção,vaso_Local da Lesão - Intervenção,vaso_Grau de estenose pré - Intervenção,vaso_Tipo de Lesão - Intervenção,vaso_Tipo de Lesão - Intrastent - Intervenção,vaso_Tipo de Lesão - Focal - Intervenção,...,vaso_Tortuosidade - Intervenção,vaso_Angulação - Intervenção,vaso_Grau de Importância - Intervenção,vaso_Bifurcação - Intervenção,vaso_Maior que 20mm - Intervenção,vaso_Trombo - Intervenção,vaso_Placa Rota - Intervenção,vaso_Placa Ulcerada - Intervenção,vaso_Ectasia - Intervenção,vaso_Aneurisma - Intervenção
0,2700,39629,UNIMED,Com lesão,CX,Médio,70.0,De Novo,NaN,NaN,...,NaN,NaN,Grande,NaN,Não,Não,Não,Não,Não,Não
1,8500,45437,S.U.S,Com lesão,Mg1,Óstio,70.0,De Novo,NaN,NaN,...,NaN,NaN,Grande,NaN,Não,Não,Não,Não,Não,Não
2,8600,45547,SÃO CRISTÓVÃO,Com lesão,DA,Médio,70.0,De Novo,NaN,NaN,...,NaN,NaN,Grande,NaN,Não,Não,Não,Não,Não,Não
3,8600,45942,S.U.S,Com lesão,DA,Distal,80.0,De Novo,NaN,NaN,...,NaN,NaN,Grande,NaN,Não,Não,Não,Não,Não,Não
4,1016,38193,PREVENT,Com lesão,CD,Proximal,95.0,De Novo,NaN,NaN,...,Leve,NaN,Grande,NaN,Não,Não,Não,Não,Não,Não
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13084,51585,69452,AMIL,Com lesão,Mg1,Proximal,70.0,De Novo,NaN,NaN,...,NaN,NaN,Grande,NaN,Não,Não,Não,Não,Não,Não
13085,51585,69452,AMIL,Com lesão,Mg2,Proximal,80.0,De Novo,NaN,NaN,...,NaN,NaN,Grande,NaN,Não,Não,Não,Não,Não,Não
13086,51624,5560,BRADESCO SAÚDE,Com lesão,CD,Médio,70.0,De Novo,NaN,NaN,...,NaN,NaN,Grande,NaN,Não,Não,Não,Não,Não,Não
13087,51808,5661,SULAMÉRICA,Com lesão,PVSVPD,Anastomose,50.0,De Novo,NaN,NaN,...,NaN,NaN,Grande,NaN,Não,Não,Não,Não,Não,Não


In [ ]:
# Select all duplicate rows based on one column
duplicateRowsDF = vaso_df[vaso_df.duplicated(['vaso_Nº da Intervenção'], keep=False)]
duplicateRowsDF = duplicateRowsDF.sort_values(by=['vaso_Nº da Intervenção'])
duplicateRowsDF

,vaso_Nº. Paciente,vaso_Nº da Intervenção,vaso_Convênio,vaso_Anatomia - Intervenção,vaso_Vaso coronário - Intervenção,vaso_Local da Lesão - Intervenção,vaso_Grau de estenose pré - Intervenção,vaso_Tipo de Lesão - Intervenção,vaso_Tipo de Lesão - Intrastent - Intervenção,vaso_Tipo de Lesão - Focal - Intervenção,...,vaso_Tortuosidade - Intervenção,vaso_Angulação - Intervenção,vaso_Grau de Importância - Intervenção,vaso_Bifurcação - Intervenção,vaso_Maior que 20mm - Intervenção,vaso_Trombo - Intervenção,vaso_Placa Rota - Intervenção,vaso_Placa Ulcerada - Intervenção,vaso_Ectasia - Intervenção,vaso_Aneurisma - Intervenção
2236,12330,4,UNIMED,Com lesão,MamáriaCD,Distal nativo,99.0,De Novo,NaN,NaN,...,NaN,NaN,Grande,NaN,Sim,Não,Não,Não,Não,Não
8079,41017,4,NUCLEN,Com lesão,DA,Proximal,70.0,De Novo,NaN,NaN,...,NaN,NaN,Grande,NaN,Não,Não,Não,Não,Não,Não
1740,1560,4,MEDIAL SAÚDE,Com lesão,CD,Médio,95.0,De Novo,NaN,NaN,...,NaN,NaN,Grande,NaN,Não,Sim,Sim,Não,Não,Não
10356,12599,7,UNIMED,Com lesão,DA,Proximal,90.0,De Novo,NaN,NaN,...,NaN,NaN,Grande,NaN,Não,Não,Não,Sim,Não,Não
3890,12599,7,UNIMED,Com lesão,CX,Médio,80.0,De Novo,NaN,NaN,...,NaN,NaN,Grande,NaN,Não,Não,Não,Não,Não,Não
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13035,46285,69416,NOTRE DAME,Com lesão,VPE,Médio,90.0,Restenose intrastent,Difusa,NaN,...,NaN,NaN,Grande,NaN,Não,Não,Não,Não,Não,Não
8179,46285,69416,NOTRE DAME,Com lesão,CD,Proximal,70.0,De Novo,NaN,NaN,...,NaN,NaN,Grande,NaN,Não,Não,Não,Não,Não,Não
1663,46285,69416,NOTRE DAME,Com lesão,CD,Médio,60.0,De Novo,NaN,NaN,...,NaN,NaN,Grande,NaN,Não,Não,Não,Não,Não,Não
13084,51585,69452,AMIL,Com lesão,Mg1,Proximal,70.0,De Novo,NaN,NaN,...,NaN,NaN,Grande,NaN,Não,Não,Não,Não,Não,Não


In [ ]:
duplicatedCompositeKeys = (vaso_df[vaso_df.duplicated(subset=['vaso_Nº. Paciente','vaso_Nº da Intervenção'], keep=False)]).sort_values(by=['vaso_Nº. Paciente'])
duplicatedCompositeKeys

,vaso_Nº. Paciente,vaso_Nº da Intervenção,vaso_Convênio,vaso_Anatomia - Intervenção,vaso_Vaso coronário - Intervenção,vaso_Local da Lesão - Intervenção,vaso_Grau de estenose pré - Intervenção,vaso_Tipo de Lesão - Intervenção,vaso_Tipo de Lesão - Intrastent - Intervenção,vaso_Tipo de Lesão - Focal - Intervenção,...,vaso_Tortuosidade - Intervenção,vaso_Angulação - Intervenção,vaso_Grau de Importância - Intervenção,vaso_Bifurcação - Intervenção,vaso_Maior que 20mm - Intervenção,vaso_Trombo - Intervenção,vaso_Placa Rota - Intervenção,vaso_Placa Ulcerada - Intervenção,vaso_Ectasia - Intervenção,vaso_Aneurisma - Intervenção
3355,1017,38128,S.U.S,Com lesão,DA,Médio,70.0,De Novo,NaN,NaN,...,NaN,NaN,Grande,"1,1,1",Não,Não,Não,Não,Não,Não
5,1017,38128,S.U.S,Com lesão,Dg1,Óstio,70.0,De Novo,NaN,NaN,...,NaN,NaN,Moderada,"1,1,1",Não,Não,Não,Não,Não,Não
1713,1057,38316,S.U.S,Com lesão,CD,Médio,80.0,De Novo,NaN,NaN,...,NaN,NaN,Grande,NaN,Não,Não,Não,Não,Não,Não
1712,1057,38316,S.U.S,Com lesão,CD,Proximal,80.0,De Novo,NaN,NaN,...,NaN,NaN,Grande,NaN,Não,Não,Não,Não,Não,Não
4987,1061,57118,UNIMED,Com lesão,DA,Médio,90.0,Restenose intrastent,Focal,Borda IB,...,NaN,NaN,Grande,NaN,Não,Não,Não,Não,Não,Não
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3349,52043,1296,PARTICULAR,Com lesão,Mg2,Proximal,80.0,De Novo,NaN,NaN,...,NaN,NaN,Moderada,NaN,Não,Não,Não,Não,Não,Não
3348,52043,1296,PARTICULAR,Com lesão,DA,Proximal,70.0,De Novo,NaN,NaN,...,NaN,NaN,Grande,NaN,Não,Não,Não,Não,Não,Não
1707,52043,1296,PARTICULAR,Com lesão,VPE,Proximal,100.0,Oclusão total crônica,NaN,NaN,...,NaN,NaN,Grande,NaN,Não,Não,Não,Não,Não,Não
3350,52223,1454,SULAMÉRICA,Com lesão,CD,Proximal,100.0,De Novo,NaN,NaN,...,NaN,NaN,Grande,NaN,Não,Sim,Não,Não,Não,Não


In [ ]:
evolucao_df = evolucao_df[evolucao_df["evolucao_Nº. Paciente"] != 19542]
paciente_df = paciente_df[paciente_df["paciente_Nº. Paciente"] != 19542]
procedimento_df = procedimento_df[procedimento_df["procedimento_Nº. Paciente"] != 19542]
complicacao_df = complicacao_df[complicacao_df["complicacao_Nº. Paciente"] != 19542]
balao_df = balao_df[balao_df["balao_Nº. Paciente"] != 19542]
stent_df = stent_df[stent_df["stent_Nº. Paciente"] != 19542]
vaso_df = vaso_df[vaso_df["vaso_Nº. Paciente"] != 19542]

# Transformation

## Evolucao

#### Operations

| Variável                               | Operação                                                    |
|----------------------------------------|-------------------------------------------------------------|
| evolucao_Nº. Paciente                  | CHAVE DE CRUZAMENTO E AGREGADOR                             |
| evolucao_Nº da Intervenção             | CHAVE DE CRUZAMENTO E AGREGADOR                             |
| evolucao_Convênio                      | X                                                           |
| evolucao_Sucesso Clínico               | enconding Sim/Não => 1/0 renomear Problema_desfecho_clinico |
| evolucao_Infarto após Intervenção      | enconding Sim/Não => 1/0                                    |
| evolucao_Tipo Infarto após Intervenção | X                                                           |
| evolucao_Óbito Hospitalar              | enconding Sim/Não => 1/0                                    |
| evolucao_Causa do Óbito Hospitalar     | X                                                           |
| evolucao_Oclusão                       | X                                                           |
| evolucao_Tratamento da Oclusão         | X                                                           |
| evolucao_Complicação Vascular          | enconding Sim/Não => 1/0                                    |
| evolucao_Tipo Complicação Vascular     | X                                                           |
| evolucao_I.R.A                         | enconding Sim/Não => 1/0                                    |
| evolucao_I.R.A Dialítica               | enconding Sim/Não => 1/0                                    |
| evolucao_A.V.C Hemorrágico             | enconding Sim/Não => 1/0                                    |
| evolucao_A.V.C Isquêmico               | enconding Sim/Não => 1/0                                    |
| evolucao_Sangramento TIMI              | X                                                           |
| evolucao_Sangramento GUSTO             | X                                                           |
| evolucao_Quantidade Vaso Tratado       | X                                                           |

In [ ]:
evolucao_df.rename(columns={"evolucao_Sucesso Clínico": "evolucao_Problema Clínico"}, inplace=True)
evolucao_df.drop(columns=['evolucao_Convênio','evolucao_Tipo Infarto após Intervenção','evolucao_Causa do Óbito Hospitalar', 'evolucao_Oclusão', 'evolucao_Tratamento da Oclusão', 'evolucao_Tipo Complicação Vascular', 'evolucao_Sangramento TIMI', 'evolucao_Sangramento GUSTO', 'evolucao_Quantidade Vaso Tratado'], inplace=True)

/home/titobr/mestrado/mestrado-venv/lib/python3.6/site-packages/pandas/core/frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/home/titobr/mestrado/mestrado-venv/lib/python3.6/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
for col in evolucao_df.columns:
    if col in ['evolucao_Nº. Paciente', 'evolucao_Nº da Intervenção']:
        continue
        
    if ('evolucao_Problema Clínico' in col):
        evolucao_df[col] = (evolucao_df[col] == 'Não').astype(int)
    else:
        evolucao_df[col] = (evolucao_df[col] == 'Sim').astype(int)

/home/titobr/mestrado/mestrado-venv/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/titobr/mestrado/mestrado-venv/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
# cols_encoding = ["evolucao_Fracasso Clínico", "evolucao_Infarto após Intervenção", "evolucao_Tipo Infarto após Intervenção", \
#                 "evolucao_Óbito Hospitalar", "evolucao_Causa do Óbito Hospitalar", "evolucao_Oclusão", \
#                 "evolucao_Tratamento da Oclusão", "evolucao_Complicação Vascular", "evolucao_Tipo Complicação Vascular",\
#                 "evolucao_I.R.A", "evolucao_I.R.A Dialítica", "evolucao_A.V.C Hemorrágico", "evolucao_A.V.C Isquêmico", \
#                 "evolucao_Sangramento TIMI", "evolucao_Sangramento GUSTO"]
# evolucao_df = pd.get_dummies(evolucao_df, columns = cols_encoding)

In [ ]:
evolucao_df.sort_values(by=['evolucao_Nº. Paciente'])

,evolucao_Nº. Paciente,evolucao_Nº da Intervenção,evolucao_Problema Clínico,evolucao_Infarto após Intervenção,evolucao_Óbito Hospitalar,evolucao_Complicação Vascular,evolucao_I.R.A,evolucao_I.R.A Dialítica,evolucao_A.V.C Hemorrágico,evolucao_A.V.C Isquêmico
8364,1000,38108,0,0,0,0,0,0,0,0
3567,1005,38117,0,0,0,0,0,0,0,0
4756,1006,38112,0,0,0,0,0,0,0,0
8365,1016,38193,0,0,0,0,0,0,0,0
4757,1017,38128,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
9637,52140,5625,0,0,0,0,0,0,0,0
1188,52223,1454,0,0,0,0,0,0,0,0
8362,52237,69579,0,0,0,0,0,0,0,0
9638,52248,5698,0,0,0,0,0,0,0,0


In [ ]:
# aggregate_params = {}
# for col in evolucao_df.columns:
#     print(col)
#     if (col not in ['evolucao_Nº. Paciente','evolucao_Nº da Intervenção']):
#         aggregate_params[col] = ['sum']

In [ ]:
# pp.pprint(aggregate_params)

In [ ]:
# evolucao_agg_df = evolucao_df.groupby(['evolucao_Nº. Paciente','evolucao_Nº da Intervenção']).agg(aggregate_params)
# evolucao_agg_df.reset_index(inplace=True)
# evolucao_agg_df

In [ ]:
print('Index length', len(evolucao_df.index))
print('Unique', len(evolucao_df.index.unique()))

Index length 9636
Unique 9636


## Paciente

#### Operations

| Variável                                      | Operação                        |
|-----------------------------------------------|---------------------------------|
| paciente_Nº. Paciente                         | CHAVE DE CRUZAMENTO E AGREGADOR |
| paciente_Sexo                                 | encoding                        |
| paciente_Idade Atual                          | numérica                        |
| paciente_Idade no procedimento                | numérica                        |
| paciente_Raça                                 | encoding                        |
| paciente_Procedência                          | encoding                        |
| paciente_Grau de Instrução                    | encoding                        |
| paciente_Faixa Etária                         | X                               |
| paciente_HAS                                  | encoding                        |
| paciente_Diabetes                             | encoding                        |
| paciente_Tratamento Diabetes                  | encoding                        |
| paciente_Dislipidemia                         | encoding                        |
| paciente_Tratamento Dislipidemia              | encoding                        |
| paciente_Tabagista                            | encoding                        |
| paciente_Cigarros (dia) - Tabagista           | encoding                        |
| paciente_Tempo                                | encoding                        |
| paciente_Ex-Tabagista                         | encoding                        |
| paciente_Cigarros (dia) - Ex-Tabagista        | encoding                        |
| paciente_Tempo de Interrupção                 | encoding                        |
| paciente_IAM Prévio                           | encoding                        |
| paciente_Síndrome Metabólica                  | encoding                        |
| paciente_Antecedentes Familiares              | encoding                        |
| paciente_Tipo Antecedentes Familiares         | encoding                        |
| paciente_IRC                                  | encoding                        |
| paciente_Tipo IRC                             | encoding                        |
| paciente_AVC prévio                           | encoding                        |
| paciente_Doença Vascular Periférica           | encoding                        |
| paciente_Tipo Doença Vascular Periférica      | encoding                        |
| paciente_História de Trombose Venosa Profunda | encoding                        |
| paciente_DPOC                                 | encoding                        |
| paciente_Alergia ao iodo                      | encoding                        |
| paciente_DAC conhecida                        | encoding                        |
| paciente_Cate Prévio                          | encoding                        |
| paciente_Cirurgia de Revascularização Prévia  | encoding                        |
| paciente_Intervenção Percutânea Prévia        | encoding                        |
| paciente_Pressão Arterial                     | 120-130 (split_min_max)         |
| paciente_Frequência Cardíaca                  | numérica                        |
| paciente_Pressão Arterial Diastólica          | 60-80 (split_min_max)           |
| paciente_Frequência Respiratória              | numérica                        |
| paciente_Temperatura (ºC)                     | numérica                        |
| paciente_Cintura Abdominal (cm)               | X                               |
| paciente_Peso (Kg)                            | numérica                        |
| paciente_Altura (metros)                      | numérica                        |
| paciente_Índice de Massa Corporal             | numérica                        |
| paciente_Creatinina Sérica                    | numérica                        |
| paciente_MDRD (ml/min/1,73m²)                 | numérica                        |
| paciente_Clearance Cr (ml/min)                | numérica                        |

Não é necessário agregar pois não há repetição de pacientes

In [ ]:
paciente_df.drop(columns=['paciente_Cintura Abdominal (cm)', 'paciente_Faixa Etária'], inplace=True)

/home/titobr/mestrado/mestrado-venv/lib/python3.6/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
new = paciente_df['paciente_Pressão Arterial'].str.split("-", n = 1, expand = True)
new[0] = new[0].str.replace('<','')
new[0] = new[0].str.replace('>','')
new[0] = new[0].str.replace('=','')
new[1] = new[1].str.replace('<','')
new[1] = new[1].str.replace('>','')
new[1] = new[1].str.replace('=','')
paciente_df['paciente_Pressão Arterial_min']= new[0] 
paciente_df['paciente_Pressão Arterial_max']= new[1]

new = paciente_df['paciente_Pressão Arterial Diastólica'].str.split("-", n = 1, expand = True)
new[0] = new[0].str.replace('<','')
new[0] = new[0].str.replace('>','')
new[0] = new[0].str.replace('=','')
new[1] = new[1].str.replace('<','')
new[1] = new[1].str.replace('>','')
new[1] = new[1].str.replace('=','')
paciente_df['paciente_Pressão Arterial Diastólica_min']= new[0] 
paciente_df['paciente_Pressão Arterial Diastólica_max']= new[1]

paciente_df.drop(columns=['paciente_Pressão Arterial', 'paciente_Pressão Arterial Diastólica'], inplace=True)

/home/titobr/mestrado/mestrado-venv/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/titobr/mestrado/mestrado-venv/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/home/titobr/mestrado/mestrado-venv/lib/python3.6/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_i

In [ ]:
cols_encoding = ['paciente_Sexo', 'paciente_Raça', 'paciente_Procedência', \
'paciente_Grau de Instrução', 'paciente_HAS', 'paciente_Diabetes', \
'paciente_Tratamento Diabetes', 'paciente_Dislipidemia', 'paciente_Tratamento Dislipidemia', \
'paciente_Tabagista', 'paciente_Cigarros (dia) - Tabagista', 'paciente_Tempo', \
'paciente_Ex-Tabagista', 'paciente_Cigarros (dia) - Ex-Tabagista', 'paciente_Tempo de Interrupção', \
'paciente_IAM Prévio', 'paciente_Síndrome Metabólica', 'paciente_Antecedentes Familiares', \
'paciente_Tipo Antecedentes Familiares', 'paciente_IRC', 'paciente_Tipo IRC', \
'paciente_AVC prévio', 'paciente_Doença Vascular Periférica', 'paciente_Tipo Doença Vascular Periférica', \
'paciente_História de Trombose Venosa Profunda', 'paciente_DPOC', 'paciente_Alergia ao iodo', \
'paciente_DAC conhecida', 'paciente_Cate Prévio', 'paciente_Cirurgia de Revascularização Prévia', \
'paciente_Intervenção Percutânea Prévia']
paciente_df = pd.get_dummies(paciente_df, columns = cols_encoding)

In [ ]:
paciente_df.sort_values(by=['paciente_Nº. Paciente'])

,paciente_Nº. Paciente,paciente_Idade Atual,paciente_Idade no procedimento,paciente_Frequência Cardíaca,paciente_Frequência Respiratória,paciente_Temperatura (ºC),paciente_Peso (Kg),paciente_Altura (metros),paciente_Índice de Massa Corporal,paciente_Creatinina Sérica,...,paciente_Alergia ao iodo_Não,paciente_Alergia ao iodo_Sim,paciente_DAC conhecida_Não,paciente_DAC conhecida_Sim,paciente_Cate Prévio_Não,paciente_Cate Prévio_Sim,paciente_Cirurgia de Revascularização Prévia_Não,paciente_Cirurgia de Revascularização Prévia_Sim,paciente_Intervenção Percutânea Prévia_Não,paciente_Intervenção Percutânea Prévia_Sim
1107,1000,90,79,NaN,NaN,NaN,79.0,1.65,29.02,NaN,...,1,0,0,1,1,0,1,0,1,0
1110,1005,79,68,NaN,NaN,NaN,63.0,1.54,26.56,NaN,...,1,0,1,0,0,1,1,0,1,0
6529,1006,97,86,NaN,NaN,NaN,56.0,1.58,22.43,NaN,...,1,0,0,1,0,1,1,0,1,0
7690,1016,76,65,64.0,NaN,NaN,58.0,1.60,22.66,NaN,...,1,0,0,1,0,1,1,0,1,0
1,1017,70,59,75.0,16.0,NaN,61.0,1.70,21.11,NaN,...,1,0,0,1,0,1,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2197,52140,59,59,NaN,NaN,NaN,79.0,1.69,27.66,NaN,...,1,0,1,0,1,0,1,0,1,0
7686,52223,59,59,NaN,NaN,NaN,107.0,1.88,30.27,NaN,...,1,0,1,0,1,0,1,0,1,0
1106,52237,57,57,NaN,NaN,NaN,100.0,1.65,36.73,NaN,...,1,0,1,0,1,0,1,0,1,0
3281,52248,75,75,67.0,20.0,36.1,82.0,1.86,23.70,1.24,...,1,0,1,0,1,0,1,0,1,0


## Procedimento

#### Operations
| Variável                                                  | Operação                                      |
|-----------------------------------------------------------|-----------------------------------------------|
| procedimento_Nº. Paciente                                 | CHAVE DE CRUZAMENTO E AGREGADOR               |
| procedimento_Nº da Intervenção                            | CHAVE DE CRUZAMENTO E AGREGADOR               |
| procedimento_Anestesia - Intervenção                      | enconding e contagem por paciente+intervenção |
| procedimento_Droga - Intervenção                          | enconding e contagem por paciente+intervenção |
| procedimento_Técnica - Intervenção                        | enconding e contagem por paciente+intervenção |
| procedimento_Introdutor - Intervenção                     | enconding e contagem por paciente+intervenção |
| procedimento_Via de acesso - Intervenção                  | enconding e contagem por paciente+intervenção |
| procedimento_Lado Via de Acesso - Intervenção             | enconding e contagem por paciente+intervenção |
| procedimento_Tipo Contraste - Intervenção                 | enconding e contagem por paciente+intervenção |
| procedimento_Nome Comercial - Intervenção                 | enconding e contagem por paciente+intervenção |
| procedimento_Tipo de Intervenção                          | enconding e contagem por paciente+intervenção |
| procedimento_Trombolítico Prévio - Primária               | enconding e contagem por paciente+intervenção |
| procedimento_Trombolítico Prévio - Resgate                | enconding e contagem por paciente+intervenção |
| procedimento_Trombolítico Prévio - Eletiva                | enconding e contagem por paciente+intervenção |
| procedimento_Trombolítico Prévio - Ad-hoc                 | enconding e contagem por paciente+intervenção |
| procedimento_TCA                                          | enconding e contagem por paciente+intervenção |
| procedimento_AAS                                          | enconding e contagem por paciente+intervenção |
| procedimento_Ticlopidina                                  | enconding e contagem por paciente+intervenção |
| procedimento_Clopidogrel                                  | enconding e contagem por paciente+intervenção |
| procedimento_HNF                                          | enconding e contagem por paciente+intervenção |
| procedimento_HBPM                                         | enconding e contagem por paciente+intervenção |
| procedimento_Abciximab                                    | enconding e contagem por paciente+intervenção |
| procedimento_Tirofiban                                    | enconding e contagem por paciente+intervenção |
| procedimento_Eptifibatide                                 | enconding e contagem por paciente+intervenção |
| procedimento_Adenosina                                    | enconding e contagem por paciente+intervenção |
| procedimento_Ultrasom Intracoronário                      | enconding e contagem por paciente+intervenção |
| procedimento_Angiografia Quantitativa                     | enconding e contagem por paciente+intervenção |
| procedimento_Inibidor de glicoproteína                    | enconding e contagem por paciente+intervenção |
| procedimento_Clopidogrel.1                                | enconding e contagem por paciente+intervenção |
| procedimento_Porta/Interv.(hrs) - Urgência                | máx, min, contagem                            |
| procedimento_Dor/ATC (horas) - Resgate                    | máx, min, contagem                            |
| procedimento_Dor/ATC (dias) - Eletiva - Ad-hoc            | máx, min, contagem                            |
| procedimento_Qtde (ml)                                    | min, max, soma, contagem                      |
| procedimento_Duração Interv. (min)                        | soma, contagem media, max, min                |
| procedimento_Dose (mg)                                    | soma, contagem media, max, min                |
| procedimento_Tempo origem/balão (minutos) - Primária      | soma, contagem, media, max, min               |
| procedimento_Tempo dor/balão (minutos) - Primária         | soma, contagem, media, max, min               |
| procedimento_Tempo Hosp. Band./balão (minutos) - Primária | soma, contagem, media, max, min               |
| procedimento_Convênio                                     | X                                             |
| procedimento_Preceptor - Intervenção                      | x                                             |
| procedimento_Operador 1 - Intervenção                     | x                                             |
| procedimento_Operador 2 - Intervenção                     | x                                             |
| procedimento_Ano da Intervenção                           | X                                             |
| procedimento_Data da Intervenção                          | X                                             |

In [ ]:
cols_to_drop = ['procedimento_Convênio', 'procedimento_Preceptor - Intervenção', 'procedimento_Operador 1 - Intervenção', \
                'procedimento_Operador 2 - Intervenção', 'procedimento_Ano da Intervenção', 'procedimento_Data da Intervenção']
procedimento_df.drop(columns=cols_to_drop, inplace=True)

/home/titobr/mestrado/mestrado-venv/lib/python3.6/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
cols_encoding = ['procedimento_Anestesia - Intervenção', 'procedimento_Droga - Intervenção', 'procedimento_Técnica - Intervenção',\
                'procedimento_Introdutor - Intervenção', 'procedimento_Via de acesso - Intervenção', \
                'procedimento_Lado Via de Acesso - Intervenção', 'procedimento_Tipo Contraste - Intervenção', \
                'procedimento_Nome Comercial - Intervenção', 'procedimento_Tipo de Intervenção', \
                'procedimento_Trombolítico Prévio - Primária', 'procedimento_Trombolítico Prévio - Resgate', \
                'procedimento_Trombolítico Prévio - Eletiva', 'procedimento_Trombolítico Prévio - Ad-hoc', 'procedimento_TCA', \
                'procedimento_AAS', 'procedimento_Ticlopidina', 'procedimento_Clopidogrel', 'procedimento_HNF', 'procedimento_HBPM',\
                'procedimento_Abciximab', 'procedimento_Tirofiban', 'procedimento_Eptifibatide', 'procedimento_Adenosina', \
                'procedimento_Ultrasom Intracoronário', 'procedimento_Angiografia Quantitativa', 'procedimento_Inibidor de glicoproteína',\
                'procedimento_Clopidogrel.1']
procedimento_df = pd.get_dummies(procedimento_df, columns = cols_encoding)

In [ ]:
procedimento_df.sort_values(by=['procedimento_Nº. Paciente'])

,procedimento_Nº. Paciente,procedimento_Nº da Intervenção,procedimento_Qtde (ml),procedimento_Porta/Interv.(hrs) - Urgência,procedimento_Dor/ATC (horas) - Resgate,procedimento_Dor/ATC (dias) - Eletiva - Ad-hoc,procedimento_Duração Interv. (min),procedimento_Dose (mg),procedimento_Tempo origem/balão (minutos) - Primária,procedimento_Tempo dor/balão (minutos) - Primária,...,procedimento_Ultrasom Intracoronário_Não,procedimento_Ultrasom Intracoronário_Sim,procedimento_Angiografia Quantitativa_Não,procedimento_Angiografia Quantitativa_Sim,procedimento_Inibidor de glicoproteína_Agrastat,procedimento_Inibidor de glicoproteína_Nenhum,procedimento_Inibidor de glicoproteína_Reo-Pro,procedimento_Clopidogrel.1_< 6h,procedimento_Clopidogrel.1_> 6h,procedimento_Clopidogrel.1_Não
7263,1000,38108,150,NaN,NaN,30.0,NaN,300,NaN,NaN,...,1,0,0,1,0,1,0,0,1,0
6091,1005,38117,80,NaN,NaN,30.0,20.0,300,NaN,NaN,...,1,0,0,1,0,1,0,1,0,0
2362,1006,38112,150,NaN,NaN,NaN,50.0,300,NaN,NaN,...,1,0,0,1,0,1,0,0,1,0
6092,1016,38193,120,36.0,NaN,NaN,40.0,300,NaN,NaN,...,1,0,0,1,0,1,0,0,1,0
2363,1017,38128,100,NaN,NaN,NaN,30.0,300,NaN,NaN,...,1,0,0,1,0,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2359,52140,5625,85,NaN,NaN,NaN,40.0,180,66.0,106.0,...,1,0,0,1,0,1,0,1,0,0
9642,52223,1454,450,NaN,NaN,NaN,90.0,0,1577.0,1680.0,...,1,0,1,0,0,0,1,1,0,0
7262,52237,69579,120,NaN,NaN,NaN,30.0,600,NaN,NaN,...,1,0,0,1,0,1,0,1,0,0
9643,52248,5698,170,NaN,NaN,NaN,50.0,75,NaN,NaN,...,1,0,1,0,0,1,0,0,1,0


In [ ]:
aggregate_params = {}
for col in procedimento_df.columns:
    print(col)
    if (col in ['procedimento_Porta/Interv.(hrs) - Urgência', 'procedimento_Dor/ATC (horas) - Resgate','procedimento_Dor/ATC (dias) - Eletiva - Ad-hoc']):
        aggregate_params[col] = ['max', 'min', 'count']  #Contagem aqui dá certo?
    elif (col in ['procedimento_Duração Interv. (min)', 'procedimento_Dose (mg)', 'procedimento_Tempo origem/balão (minutos) - Primária', 'procedimento_Tempo dor/balão (minutos) - Primária', 'procedimento_Tempo Hosp. Band./balão (minutos) - Primária']):
        aggregate_params[col] = ['max', 'min', 'mean', 'sum', 'count']
    elif (col in ['procedimento_Qtde (ml)']):
        aggregate_params[col] = ['max', 'min', 'sum', 'count']
    for col_encoding in cols_encoding:
        if (col_encoding in col):
            aggregate_params[col] = ['sum']

procedimento_Nº. Paciente
procedimento_Nº da Intervenção
procedimento_Qtde (ml)
procedimento_Porta/Interv.(hrs) - Urgência
procedimento_Dor/ATC (horas) - Resgate
procedimento_Dor/ATC (dias) - Eletiva - Ad-hoc
procedimento_Duração Interv. (min)
procedimento_Dose (mg)
procedimento_Tempo origem/balão (minutos) - Primária
procedimento_Tempo dor/balão (minutos) - Primária
procedimento_Tempo Hosp. Band./balão (minutos) - Primária
procedimento_Anestesia - Intervenção_Geral Inalatória
procedimento_Anestesia - Intervenção_Geral Sedação
procedimento_Anestesia - Intervenção_Local
procedimento_Droga - Intervenção_Fentanil
procedimento_Droga - Intervenção_Midazolan
procedimento_Droga - Intervenção_Propafenona
procedimento_Droga - Intervenção_Thionembutal
procedimento_Droga - Intervenção_Xilocaína 2%
procedimento_Técnica - Intervenção_Convencional
procedimento_Técnica - Intervenção_kissing-balloon
procedimento_Introdutor - Intervenção_10F
procedimento_Introdutor - Intervenção_11F
procedimento_Introd

In [ ]:
pp.pprint(aggregate_params)

{   'procedimento_AAS_Não': ['sum'],
    'procedimento_AAS_Sim': ['sum'],
    'procedimento_Abciximab_Não': ['sum'],
    'procedimento_Abciximab_Sim': ['sum'],
    'procedimento_Adenosina_Não': ['sum'],
    'procedimento_Adenosina_Sim': ['sum'],
    'procedimento_Anestesia - Intervenção_Geral Inalatória': ['sum'],
    'procedimento_Anestesia - Intervenção_Geral Sedação': ['sum'],
    'procedimento_Anestesia - Intervenção_Local': ['sum'],
    'procedimento_Angiografia Quantitativa_Não': ['sum'],
    'procedimento_Angiografia Quantitativa_Sim': ['sum'],
    'procedimento_Clopidogrel.1_< 6h': ['sum'],
    'procedimento_Clopidogrel.1_> 6h': ['sum'],
    'procedimento_Clopidogrel.1_Não': ['sum'],
    'procedimento_Clopidogrel_Não': ['sum'],
    'procedimento_Clopidogrel_Sim': ['sum'],
    'procedimento_Dor/ATC (dias) - Eletiva - Ad-hoc': ['max', 'min', 'count'],
    'procedimento_Dor/ATC (horas) - Resgate': ['max', 'min', 'count'],
    'procedimento_Dose (mg)': ['max', 'min', 'mean', 'sum',

In [ ]:
procedimento_agg_df = procedimento_df.groupby(['procedimento_Nº. Paciente','procedimento_Nº da Intervenção']).agg(aggregate_params)
procedimento_agg_df.reset_index(inplace=True)
procedimento_agg_df

procedimento_Nº. Paciente procedimento_Nº da Intervenção  \
                                                                
0                         1000                          38108   
1                         1005                          38117   
2                         1006                          38112   
3                         1016                          38193   
4                         1017                          38128   
...                        ...                            ...   
9624                     52140                           5625   
9625                     52223                           1454   
9626                     52237                          69579   
9627                     52248                           5698   
9628                     52370                           5662   

     procedimento_Qtde (ml)                  \
                        max  min  sum count   
0                       150  150  150     1   
1                        80   80   80     1   
2                       150  150  150     1   
3                       120  120  120     1   
4                       100  100  100     1   
...                     ...  ...  ...   ...   
9624                     85   85   85     1   
9625                    450  450  450     1   
9626                    120  120  120     1   
9627                    170  170  170     1   
9628                    200  200  200     1   

     procedimento_Porta/Interv.(hrs) - Urgência              \
                                            max   min count   
0                                           NaN   NaN     0   
1                                           NaN   NaN     0   
2                                           NaN   NaN     0   
3                                          36.0  36.0     1   
4                                           NaN   NaN     0   
...                                         ...   ...   ...   
9624                                        NaN   NaN     0   
9625                                        NaN   NaN     0   
9626                                        NaN   NaN     0   
9627                                        NaN   NaN     0   
9628                                        1.0   1.0     1   

     procedimento_Dor/ATC (horas) - Resgate  ...  \
                                        max  ...   
0                                       NaN  ...   
1                                       NaN  ...   
2                                       NaN  ...   
3                                       NaN  ...   
4                                       NaN  ...   
...                                     ...  ...   
9624                                    NaN  ...   
9625                                    NaN  ...   
9626                                    NaN  ...   
9627                                    NaN  ...   
9628                                    NaN  ...   

     procedimento_Ultrasom Intracoronário_Não  \
                                          sum   
0                                           1   
1                                           1   
2                                           1   
3                                           1   
4                                           1   
...                                       ...   
9624                                        1   
9625                                        1   
9626                                        1   
9627                                        1   
9628                                        1   

     procedimento_Ultrasom Intracoronário_Sim  \
                                          sum   
0                                           0   
1                                           0   
2                                           0   
3                                           0   
4                                           0   
...                                       ...   
9624                

In [ ]:
print('Index length', len(procedimento_agg_df.index))
print('Unique', len(procedimento_agg_df.index.unique()))

Index length 9629
Unique 9629


## Complicação

#### Operations
| Variável                       | Operação                        |
|--------------------------------|---------------------------------|
| complicacao_Nº. Paciente       | CHAVE DE CRUZAMENTO E AGREGADOR |
| complicacao_Nº da Intervenção  | CHAVE DE CRUZAMENTO E AGREGADOR |
| complicacao_Quando             | enconding e contagem            |
| complicacao_Destino Alta       | enconding e contagem            |
| complicacao_Óbito              | enconding e contagem            |
| complicacao_Tipo Complicação   | enconding e contagem            |
| complicacao_Grau - Complicação | enconding e contagem            |
| complicacao_Conduta            | enconding e contagem            |
| complicacao_Procedimento       | X - sem variabilidade           |
| complicacao_Contato Realizado  | X - sem variabilidade           |
| complicacao_Complicação        | X - sem variabilidade           |

# Tenho dúvida sobre a coluna Óbito.  Me parece mais um output do que uma feature

In [ ]:
cols_to_drop = ['complicacao_Procedimento', 'complicacao_Contato Realizado', 'complicacao_Complicação']
complicacao_df.drop(columns=cols_to_drop, inplace=True)

In [ ]:
cols_encoding = ['complicacao_Quando', 'complicacao_Destino Alta', 'complicacao_Óbito',\
                'complicacao_Tipo Complicação', 'complicacao_Grau - Complicação', \
                'complicacao_Conduta']
complicacao_df = pd.get_dummies(complicacao_df, columns = cols_encoding)

In [ ]:
complicacao_df.sort_values(by=['complicacao_Nº. Paciente'])

,complicacao_Nº. Paciente,complicacao_Nº da Intervenção,complicacao_Quando_Até 1 semana,complicacao_Quando_Até 24 horas,complicacao_Quando_Intra-exame até a alta,complicacao_Quando_Recuperação,complicacao_Quando_Sala de exame,complicacao_Destino Alta_Domiciliar,complicacao_Destino Alta_Enfermaria,complicacao_Destino Alta_Hospital Origem,...,complicacao_Tipo Complicação_Isquêmica,complicacao_Tipo Complicação_Neurológica,complicacao_Tipo Complicação_R. Contraste,complicacao_Tipo Complicação_Vagal,complicacao_Tipo Complicação_Vascular,complicacao_Grau - Complicação_Grave,complicacao_Grau - Complicação_Leve,complicacao_Grau - Complicação_Moderado,complicacao_Conduta_Orientação Telefônica,complicacao_Conduta_Retorno para avaliação Médica
0,1006,38112,0,0,1,0,0,1,0,0,...,0,0,0,0,1,0,0,1,0,0
286,1006,38112,1,0,0,0,0,0,0,0,...,0,0,0,0,1,0,1,0,0,0
552,1057,38316,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,0
702,1142,38727,0,0,1,0,0,1,0,0,...,1,0,0,0,0,1,0,0,0,0
136,1142,38727,0,0,1,0,0,0,1,0,...,1,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
698,51133,5485,0,0,0,0,1,0,0,0,...,1,0,0,0,0,1,0,0,0,0
699,51479,5533,0,0,0,0,1,0,0,0,...,0,0,0,0,1,1,0,0,0,0
700,51479,5533,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
835,51552,5548,0,1,0,0,0,0,0,0,...,1,0,0,0,0,1,0,0,1,0


In [ ]:
aggregate_params = {}
for col in complicacao_df.columns:
    print(col)
    if (col not in ['complicacao_Nº. Paciente', 'complicacao_Nº da Intervenção']):
        aggregate_params[col] = ['sum']

complicacao_Nº. Paciente
complicacao_Nº da Intervenção
complicacao_Quando_Até 1 semana
complicacao_Quando_Até 24 horas
complicacao_Quando_Intra-exame até a alta
complicacao_Quando_Recuperação
complicacao_Quando_Sala de exame
complicacao_Destino Alta_Domiciliar
complicacao_Destino Alta_Enfermaria
complicacao_Destino Alta_Hospital Origem
complicacao_Óbito_Não
complicacao_Óbito_Sim
complicacao_Tipo Complicação_Alérgica
complicacao_Tipo Complicação_Arritmia
complicacao_Tipo Complicação_Congestiva
complicacao_Tipo Complicação_Embólica
complicacao_Tipo Complicação_Hematoma
complicacao_Tipo Complicação_Isquêmica
complicacao_Tipo Complicação_Neurológica
complicacao_Tipo Complicação_R. Contraste
complicacao_Tipo Complicação_Vagal
complicacao_Tipo Complicação_Vascular
complicacao_Grau - Complicação_Grave
complicacao_Grau - Complicação_Leve
complicacao_Grau - Complicação_Moderado
complicacao_Conduta_Orientação Telefônica
complicacao_Conduta_Retorno para avaliação Médica


In [ ]:
pp.pprint(aggregate_params)

{   'complicacao_Conduta_Orientação Telefônica': ['sum'],
    'complicacao_Conduta_Retorno para avaliação Médica': ['sum'],
    'complicacao_Destino Alta_Domiciliar': ['sum'],
    'complicacao_Destino Alta_Enfermaria': ['sum'],
    'complicacao_Destino Alta_Hospital Origem': ['sum'],
    'complicacao_Grau - Complicação_Grave': ['sum'],
    'complicacao_Grau - Complicação_Leve': ['sum'],
    'complicacao_Grau - Complicação_Moderado': ['sum'],
    'complicacao_Quando_Até 1 semana': ['sum'],
    'complicacao_Quando_Até 24 horas': ['sum'],
    'complicacao_Quando_Intra-exame até a alta': ['sum'],
    'complicacao_Quando_Recuperação': ['sum'],
    'complicacao_Quando_Sala de exame': ['sum'],
    'complicacao_Tipo Complicação_Alérgica': ['sum'],
    'complicacao_Tipo Complicação_Arritmia': ['sum'],
    'complicacao_Tipo Complicação_Congestiva': ['sum'],
    'complicacao_Tipo Complicação_Embólica': ['sum'],
    'complicacao_Tipo Complicação_Hematoma': ['sum'],
    'complicacao_Tipo Complicaçã

In [ ]:
complicacao_agg_df = complicacao_df.groupby(['complicacao_Nº. Paciente','complicacao_Nº da Intervenção']).agg(aggregate_params)
complicacao_agg_df.reset_index(inplace=True)
complicacao_agg_df

,complicacao_Nº. Paciente,complicacao_Nº da Intervenção,complicacao_Quando_Até 1 semana,complicacao_Quando_Até 24 horas,complicacao_Quando_Intra-exame até a alta,complicacao_Quando_Recuperação,complicacao_Quando_Sala de exame,complicacao_Destino Alta_Domiciliar,complicacao_Destino Alta_Enfermaria,complicacao_Destino Alta_Hospital Origem,...,complicacao_Tipo Complicação_Isquêmica,complicacao_Tipo Complicação_Neurológica,complicacao_Tipo Complicação_R. Contraste,complicacao_Tipo Complicação_Vagal,complicacao_Tipo Complicação_Vascular,complicacao_Grau - Complicação_Grave,complicacao_Grau - Complicação_Leve,complicacao_Grau - Complicação_Moderado,complicacao_Conduta_Orientação Telefônica,complicacao_Conduta_Retorno para avaliação Médica
,,,sum,sum,sum,sum,sum,sum,sum,sum,...,sum,sum,sum,sum,sum,sum,sum,sum,sum,sum
0,1006,38112,1,0,1,0,0,1,0,0,...,0,0,0,0,2,0,1,1,0,0
1,1057,38316,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,0
2,1142,38727,0,0,2,0,0,1,1,0,...,2,0,0,0,0,2,0,0,0,0
3,1152,38280,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,0
4,1208,38346,0,0,1,0,0,0,0,1,...,0,0,0,1,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1025,50862,5455,0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1026,51133,5485,0,0,0,0,1,0,0,0,...,1,0,0,0,0,1,0,0,0,0
1027,51479,5533,0,0,0,0,2,0,0,0,...,0,0,0,0,1,1,0,1,0,0


In [ ]:
print('Index length', len(complicacao_agg_df.index))
print('Unique', len(complicacao_agg_df.index.unique()))

Index length 1030
Unique 1030


## Balão

#### Operations
| Variável                                           | Operação                                         |
|----------------------------------------------------|--------------------------------------------------|
| balao_Nº. Paciente                                 | CHAVE DE CRUZAMENTO E AGREGADOR                  |
| balao_Nº da Intervenção                            | CHAVE DE CRUZAMENTO E AGREGADOR                  |
| balao_Kissing wire - Balão                         | completar vazio com não                          |
| balao_Diâmetro Cateter - Balão                     | contagem, média, máx e min por intervenção       |
| balao_Device Adjunto - Balão                       | enconding por categoria - e contagem em cada uma |
| balao_Resultado Angiográfico - Balão               | enconding por categoria - e contagem em cada uma |
| balao_Tipo - Dissecção - Balão                     | enconding por categoria - e contagem em cada uma |
| balao_Classificação da Lesão (ACC/AHA) - Balão     | enconding por categoria - e contagem em cada uma |
| balao_Distúrbio de Fluxo - Balão                   | enconding por categoria - e contagem em cada uma |
| balao_Tipo do Distúrbio de Fluxo - Balão           | enconding por categoria - e contagem em cada uma |
| balao_Tipo No-Reflow                               | enconding por categoria - e contagem em cada uma |
| balao_Vaso coronário - Balão                       | enconding por intervenção                        |
| balao_Kissing balloon - Balão                      | juntar com a categoria anterior                  |
| balao_Comprimento Cateter - Balão                  | média, máx e min por intervenção                 |
| balao_Pressão Final (ATM)                          | média, máx e min por intervenção                 |
| balao_Grau de estenose pós                         | média, máx e min por intervenção                 |
| balao_Timi pós                                     | média, máx e min por intervenção                 |
| balao_Blush Miocárdico                             | média, máx e min por intervenção                 |
| balao_Total Injetado (mg) - Adenosina - Balão      | soma                                             |
| balao_Total Injetado (mg) - Papaverina - Balão     | soma                                             |
| balao_Total Injetado (mg) - Nitroglicerina - Balão | soma                                             |
| balao_Total Injetado (mg) - Monocordil - Balão     | soma                                             |
| balao_Total Injetado (mg) - Nitroprussiato - Balão | soma                                             |
| balao_Total Injetado (mg) - Adrenalina - Balão     | soma                                             |
| balao_N. Injeções - Adenosina - Balão              | soma por intervenção                             |
| balao_N. Injeções - Nitroglicerina - Balão         | soma por intervenção                             |
| balao_N. Injeções - Monocordil - Balão             | soma por intervenção                             |
| balao_N. Injeções - Nitroprussiato - Balão         | soma por intervenção                             |
| balao_N. Injeções - Adrenalina - Balão             | soma por intervenção                             |
| balao_N. Injeções - Papaverina - Balão             | soma por paciente                                |
| balao_Convênio                                     | X                                                |
| balao_Adenosina - Balão                            | X                                                |
| balao_Dose/Injeção (mg) - Adenosina - Balão        | X                                                |
| balao_Papaverina - Balão                           | X                                                |
| balao_Dose/Injeção (mg) - Papaverina - Balão       | X                                                |
| balao_Nitroglicerina - Balão                       | X                                                |
| balao_Dose/Injeção (mg) - Nitroglicerina - Balão   | X                                                |
| balao_Monocordil - Balão                           | X                                                |
| balao_Dose/Injeção (mg) - Monocordil - Balão       | X                                                |
| balao_Nitroprussiato - Balão                       | X                                                |
| balao_Dose/Injeção (mg) - Nitroprussiato - Balão   | X                                                |
| balao_Adrenalina - Balão                           | X                                                |
| balao_Dose/Injeção (mg) - Adrenalina - Balão       | X                                                |

In [ ]:
cols_to_drop = ['balao_Convênio', 'balao_Adenosina - Balão', 'balao_Dose/Injeção (mg) - Adenosina - Balão', \
                'balao_Papaverina - Balão', 'balao_Dose/Injeção (mg) - Papaverina - Balão', 'balao_Nitroglicerina - Balão', \
                'balao_Monocordil - Balão', 'balao_Dose/Injeção (mg) - Monocordil - Balão', 'balao_Nitroprussiato - Balão', \
                'balao_Dose/Injeção (mg) - Nitroprussiato - Balão', 'balao_Adrenalina - Balão', 'balao_Dose/Injeção (mg) - Adrenalina - Balão']
balao_df.drop(columns=cols_to_drop, inplace=True)

In [ ]:
# balao_df.fillna(value={'balao_Kissing wire - Balão': 'Não'}, inplace=True)

In [ ]:
cols_encoding = ['balao_Device Adjunto - Balão', 'balao_Resultado Angiográfico - Balão', 'balao_Tipo - Dissecção - Balão', \
                 'balao_Classificação da Lesão (ACC/AHA) - Balão', 'balao_Distúrbio de Fluxo - Balão', 'balao_Tipo do Distúrbio de Fluxo - Balão', \
                 'balao_Tipo No-Reflow', 'balao_Vaso coronário - Balão', 'balao_Kissing wire - Balão', 'balao_Kissing balloon - Balão']
balao_df = pd.get_dummies(balao_df, columns = cols_encoding)

In [ ]:
balao_df.sort_values(by=['balao_Nº. Paciente'])

,balao_Nº. Paciente,balao_Nº da Intervenção,balao_Diâmetro Cateter - Balão,balao_Comprimento Cateter - Balão,balao_Pressão Final (ATM),balao_Grau de estenose pós,balao_Timi pós,balao_Blush Miocárdico,balao_N. Injeções - Adenosina - Balão,balao_Total Injetado (mg) - Adenosina - Balão,...,balao_Vaso coronário - Balão_PVSMg1,balao_Vaso coronário - Balão_PVSMg2,balao_Vaso coronário - Balão_RadialDg,balao_Vaso coronário - Balão_TCE,balao_Vaso coronário - Balão_VPD,balao_Vaso coronário - Balão_VPE,balao_Kissing wire - Balão_Não,balao_Kissing wire - Balão_Sim,balao_Kissing balloon - Balão_Não,balao_Kissing balloon - Balão_Sim
135,1000,38108,2.50,12.0,NaN,99.0,2.0,NaN,NaN,NaN,...,0,0,0,0,0,0,1,0,1,0
1016,1017,38128,2.50,12.0,NaN,10.0,3.0,NaN,NaN,NaN,...,0,0,0,0,0,0,1,0,0,1
136,1046,38157,2.50,12.0,8.0,40.0,3.0,NaN,NaN,NaN,...,0,0,0,0,0,0,1,0,1,0
137,1096,38210,2.50,12.0,NaN,100.0,0.0,NaN,NaN,NaN,...,0,0,0,0,0,0,1,0,1,0
1017,1107,38224,2.50,10.0,12.0,100.0,0.0,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
409,51197,1391,2.50,15.0,14.0,70.0,3.0,NaN,1.0,2.0,...,0,0,0,0,0,0,0,0,0,0
267,51479,5533,3.50,15.0,1.0,100.0,0.0,0.0,NaN,NaN,...,0,0,0,0,1,0,0,0,0,0
707,51525,5550,2.25,12.0,12.0,0.0,3.0,3.0,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
410,51574,5562,2.25,15.0,10.0,30.0,3.0,3.0,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
aggregate_params = {}
for col in balao_df.columns:
    print(col)
    if (col in ['balao_Diâmetro Cateter - Balão']):
        aggregate_params[col] = ['count', 'mean', 'max', 'min']
    elif (col in ['balao_Comprimento Cateter - Balão', 'balao_Pressão Final (ATM)', 'balao_Grau de estenose pós', 'balao_Timi pós', 'balao_Blush Miocárdico']):
        aggregate_params[col] = ['mean', 'max', 'min']
    elif (col in ['balao_Total Injetado (mg) - Adenosina - Balão', 'balao_Total Injetado (mg) - Papaverina - Balão', 'balao_Total Injetado (mg) - Nitroglicerina - Balão', 'balao_Total Injetado (mg) - Monocordil - Balão', 'balao_Total Injetado (mg) - Nitroprussiato - Balão', 'balao_Total Injetado (mg) - Adrenalina - Balão']):
        aggregate_params[col] = ['sum']
    elif (col in ['balao_N. Injeções - Adenosina - Balão', 'balao_N. Injeções - Nitroglicerina - Balão', 'balao_N. Injeções - Monocordil - Balão', 'balao_N. Injeções - Nitroprussiato - Balão', 'balao_N. Injeções - Adrenalina - Balão', 'balao_N. Injeções - Papaverina - Balão']):
        aggregate_params[col] = ['sum']
    for col_encoding in cols_encoding:
        if (col_encoding in col):
            aggregate_params[col] = ['sum']

balao_Nº. Paciente
balao_Nº da Intervenção
balao_Diâmetro Cateter - Balão
balao_Comprimento Cateter - Balão
balao_Pressão Final (ATM)
balao_Grau de estenose pós
balao_Timi pós
balao_Blush Miocárdico
balao_N. Injeções - Adenosina - Balão
balao_Total Injetado (mg) - Adenosina - Balão
balao_N. Injeções - Papaverina - Balão
balao_Total Injetado (mg) - Papaverina - Balão
balao_N. Injeções - Nitroglicerina - Balão
balao_Dose/Injeção (mg) - Nitroglicerina - Balão
balao_Total Injetado (mg) - Nitroglicerina - Balão
balao_N. Injeções - Monocordil - Balão
balao_Total Injetado (mg) - Monocordil - Balão
balao_N. Injeções - Nitroprussiato - Balão
balao_Total Injetado (mg) - Nitroprussiato - Balão
balao_N. Injeções - Adrenalina - Balão
balao_Total Injetado (mg) - Adrenalina - Balão
balao_Device Adjunto - Balão_Braquiterapia
balao_Device Adjunto - Balão_Exciser
balao_Resultado Angiográfico - Balão_I1 - Não ultrapassou a lesão
balao_Resultado Angiográfico - Balão_I2 - Ultrapassou e não dilatou
balao_Re

In [ ]:
pp.pprint(aggregate_params)

{   'balao_Blush Miocárdico': ['mean', 'max', 'min'],
    'balao_Classificação da Lesão (ACC/AHA) - Balão_A': ['sum'],
    'balao_Classificação da Lesão (ACC/AHA) - Balão_B1': ['sum'],
    'balao_Classificação da Lesão (ACC/AHA) - Balão_B2': ['sum'],
    'balao_Classificação da Lesão (ACC/AHA) - Balão_C': ['sum'],
    'balao_Comprimento Cateter - Balão': ['mean', 'max', 'min'],
    'balao_Device Adjunto - Balão_Braquiterapia': ['sum'],
    'balao_Device Adjunto - Balão_Exciser': ['sum'],
    'balao_Distúrbio de Fluxo - Balão_Não': ['sum'],
    'balao_Distúrbio de Fluxo - Balão_Sim': ['sum'],
    'balao_Diâmetro Cateter - Balão': ['count', 'mean', 'max', 'min'],
    'balao_Grau de estenose pós': ['mean', 'max', 'min'],
    'balao_Kissing balloon - Balão_Não': ['sum'],
    'balao_Kissing balloon - Balão_Sim': ['sum'],
    'balao_Kissing wire - Balão_Não': ['sum'],
    'balao_Kissing wire - Balão_Sim': ['sum'],
    'balao_N. Injeções - Adenosina - Balão': ['sum'],
    'balao_N. Injeções -

In [ ]:
balao_agg_df = balao_df.groupby(['balao_Nº. Paciente','balao_Nº da Intervenção']).agg(aggregate_params)
balao_agg_df.reset_index(inplace=True)
balao_agg_df

balao_Nº. Paciente balao_Nº da Intervenção  \
                                                  
0                  1000                   38108   
1                  1017                   38128   
2                  1046                   38157   
3                  1096                   38210   
4                  1107                   38224   
...                 ...                     ...   
1064              51197                    1391   
1065              51479                    5533   
1066              51525                    5550   
1067              51574                    5562   
1068              52223                    1454   

     balao_Diâmetro Cateter - Balão                    \
                              count  mean   max   min   
0                                 1  2.50  2.50  2.50   
1                                 1  2.50  2.50  2.50   
2                                 1  2.50  2.50  2.50   
3                                 1  2.50  2.50  2.50   
4                                 1  2.50  2.50  2.50   
...                             ...   ...   ...   ...   
1064                              1  2.50  2.50  2.50   
1065                              1  3.50  3.50  3.50   
1066                              1  2.25  2.25  2.25   
1067                              1  2.25  2.25  2.25   
1068                              1  2.50  2.50  2.50   

     balao_Comprimento Cateter - Balão             balao_Pressão Final (ATM)  \
                                  mean   max   min                      mean   
0                                 12.0  12.0  12.0                       NaN   
1                                 12.0  12.0  12.0                       NaN   
2                                 12.0  12.0  12.0                       8.0   
3                                 12.0  12.0  12.0                       NaN   
4                                 10.0  10.0  10.0                      12.0   
...                                ...   ...   ...                       ...   
1064                              15.0  15.0  15.0                      14.0   
1065                              15.0  15.0  15.0                       1.0   
1066                              12.0  12.0  12.0                      12.0   
1067                              15.0  15.0  15.0                      10.0   
1068                              15.0  15.0  15.0                       0.0   

      ... balao_Vaso coronário - Balão_PVSMg1  \
      ...                                 sum   
0     ...                                   0   
1     ...                                   0   
2     ...                                   0   
3     ...                                   0   
4     ...                                   0   
...   ...                                 ...   
1064  ...                                   0   
1065  ...                                   0   
1066  ...                                   0   
1067  ...                                   0   
1068  ...                                   0   

     balao_Vaso coronário - Balão_PVSMg2  \
                                     sum   
0                                      0   
1                                      0   
2                                      0   
3                                      0   
4                                      0   
...                                  ...   
1064                                   0   
1065                                   0   
1066                                   0   
1067                                   0   
1068                                   0   

     balao_Vaso coronário - Balão_RadialDg balao_Vaso coronário - Balão_TCE  \
                                       sum                              sum   
0                                        0                                0   
1                                        0                                0   
2                              

In [ ]:
print('Index length', len(balao_agg_df.index))
print('Unique', len(balao_agg_df.index.unique()))

Index length 1069
Unique 1069


## Stent

#### Operations
| Variável                                           | Operação                                      |
|----------------------------------------------------|-----------------------------------------------|
| stent_Nº. Paciente                                 | CHAVE DE CRUZAMENTO E AGREGADOR               |
| stent_Nº da Intervenção                            | CHAVE DE CRUZAMENTO E AGREGADOR               |
| stent_Vaso coronário - Stent                       | enconding e contagem por paciente+intervenção |
| stent_Stent                                        | enconding e contagem por paciente+intervenção |
| stent_Estratégia                                   | enconding e contagem por paciente+intervenção |
| stent_Resultado Angiográfico - Stent               | enconding e contagem por paciente+intervenção |
| stent_Tipo - Dissecção - Stent                     | enconding e contagem por paciente+intervenção |
| stent_Localização da Dissecção                     | enconding e contagem por paciente+intervenção |
| stent_Classificação da Lesão (ACC/AHA) - Stent     | enconding e contagem por paciente+intervenção |
| stent_Kissing wire - Stent                         | enconding e contagem por paciente+intervenção |
| stent_Kissing balloon - Stent                      | enconding e contagem por paciente+intervenção |
| stent_Sobreposição                                 | enconding e contagem por paciente+intervenção |
| stent_Distúrbio de Fluxo - Stent                   | enconding e contagem por paciente+intervenção |
| stent_Tipo do Distúrbio de Fluxo - Stent           | enconding e contagem por paciente+intervenção |
| stent_Tipo No-Reflow                               | enconding e contagem por paciente+intervenção |
| stent_Técnica de bifurcação                        | enconding e contagem por paciente+intervenção |
| stent_Stent Dedicado                               | enconding e contagem por paciente+intervenção |
| stent_Adenosina - Stent                            | enconding e contagem por paciente+intervenção |
| stent_Papaverina - Stent                           | enconding e contagem por paciente+intervenção |
| stent_Nitroglicerina - Stent                       | enconding e contagem por paciente+intervenção |
| stent_Monocordil - Stent                           | enconding e contagem por paciente+intervenção |
| stent_Nitroprussiato - Stent                       | enconding e contagem por paciente+intervenção |
| stent_Adrenalina - Stent                           | enconding e contagem por paciente+intervenção |
| stent_Comprimento Dispositivo - Stent              | media, max, min                               |
| stent_Diâmetro Cateter Balão Adjunto Pré           | media, max, min                               |
| stent_Diâmetro Cateter Balão Adjunto Pós           | media, max, min                               |
| stent_Pressão Final de Liberação (ATM)             | media, max, min                               |
| stent_Grau de estenose pós                         | media, max, min                               |
| stent_Timi pós                                     | media, max, min                               |
| stent_Blush Miocárdico                             | media, max, min                               |
| stent_Diâmetro Dispositivo - Stent                 | media, max, min, contagem                     |
| stent_N. Injeções - Adenosina - Stent              | soma                                          |
| stent_Total Injetado (mg) - Adenosina - Stent      | soma                                          |
| stent_N. Injeções - Papaverina - Stent             | soma                                          |
| stent_Total Injetado (mg) - Papaverina - Stent     | soma                                          |
| stent_N. Injeções - Nitroglicerina - Stent         | soma                                          |
| stent_Total Injetado (mg) - Nitroglicerina - Stent | soma                                          |
| stent_N. Injeções - Monocordil - Stent             | soma                                          |
| stent_Total Injetado (mg) - Monocordil - Stent     | soma                                          |
| stent_N. Injeções - Nitroprussiato - Stent         | soma                                          |
| stent_Total Injetado (mg) - Nitroprussiato - Stent | soma                                          |
| stent_N. Injeções - Adrenalina - Stent             | soma                                          |
| stent_Total Injetado (mg) - Adrenalina - Stent     | soma                                          |
| stent_Convênio                                     | x                                             |
| stent_Dose/Injeção (mg) - Adenosina - Stent        | X                                             |
| stent_Dose/Injeção (mg) - Papaverina - Stent       | X                                             |
| stent_Dose/Injeção (mg) - Nitroglicerina - Stent   | X                                             |
| stent_Dose/Injeção (mg) - Monocordil - Stent       | X                                             |
| stent_Dose/Injeção (mg) - Nitroprussiato - Stent   | X                                             |
| stent_Dose/Injeção (mg) - Adrenalina - Stent       | X                                             |

In [ ]:
cols_to_drop = ['stent_Convênio', 'stent_Dose/Injeção (mg) - Adenosina - Stent', 'stent_Dose/Injeção (mg) - Papaverina - Stent', \
                'stent_Dose/Injeção (mg) - Nitroglicerina - Stent', 'stent_Dose/Injeção (mg) - Monocordil - Stent', \
                'stent_Dose/Injeção (mg) - Nitroprussiato - Stent', 'stent_Dose/Injeção (mg) - Adrenalina - Stent']
stent_df.drop(columns=cols_to_drop, inplace=True)

/home/titobr/mestrado/mestrado-venv/lib/python3.6/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
cols_encoding = ['stent_Vaso coronário - Stent', 'stent_Stent', 'stent_Estratégia', \
                 'stent_Resultado Angiográfico - Stent', 'stent_Tipo - Dissecção - Stent', 'stent_Localização da Dissecção', \
                 'stent_Classificação da Lesão (ACC/AHA) - Stent', 'stent_Kissing wire - Stent', 'stent_Kissing balloon - Stent', \
                 'stent_Sobreposição', 'stent_Distúrbio de Fluxo - Stent', 'stent_Tipo do Distúrbio de Fluxo - Stent', \
                 'stent_Tipo No-Reflow', 'stent_Técnica de bifurcação', 'stent_Stent Dedicado', 'stent_Adenosina - Stent', \
                 'stent_Papaverina - Stent', 'stent_Nitroglicerina - Stent', 'stent_Monocordil - Stent', 'stent_Nitroprussiato - Stent', \
                 'stent_Adrenalina - Stent']
stent_df = pd.get_dummies(stent_df, columns = cols_encoding)

In [ ]:
stent_df.sort_values(by=['stent_Nº. Paciente'])

,stent_Nº. Paciente,stent_Nº da Intervenção,stent_Diâmetro Dispositivo - Stent,stent_Comprimento Dispositivo - Stent,stent_Diâmetro Cateter Balão Adjunto Pré,stent_Diâmetro Cateter Balão Adjunto Pós,stent_Pressão Final de Liberação (ATM),stent_Grau de estenose pós,stent_Timi pós,stent_Blush Miocárdico,...,stent_Adenosina - Stent_Sim,stent_Papaverina - Stent_Não,stent_Papaverina - Stent_Sim,stent_Nitroglicerina - Stent_Não,stent_Nitroglicerina - Stent_Sim,stent_Monocordil - Stent_Não,stent_Monocordil - Stent_Sim,stent_Nitroprussiato - Stent_Não,stent_Adrenalina - Stent_Não,stent_Adrenalina - Stent_Sim
10846,1005,38117,3.0,8,NaN,3.5,18.0,0.0,3.0,NaN,...,0,1,0,1,0,1,0,1,1,0
4635,1006,38112,2.5,12,2.5,2.5,12.0,0.0,3.0,NaN,...,0,1,0,1,0,1,0,1,1,0
9275,1016,38193,3.0,16,2.5,NaN,12.0,0.0,3.0,NaN,...,0,1,0,1,0,1,0,1,1,0
4636,1017,38128,3.0,16,NaN,NaN,14.0,70.0,3.0,NaN,...,0,1,0,1,0,1,0,1,1,0
1541,1018,38126,3.0,28,NaN,3.0,18.0,0.0,3.0,NaN,...,0,1,0,1,0,1,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10844,52140,5625,3.5,28,NaN,3.5,18.0,0.0,3.0,3.0,...,1,1,0,1,0,0,1,1,1,0
6163,52223,1454,3.0,38,2.5,3.0,12.0,0.0,3.0,NaN,...,0,1,0,1,0,1,0,1,1,0
4631,52237,69579,3.0,16,NaN,3.5,11.0,0.0,3.0,3.0,...,0,1,0,1,0,1,0,1,1,0
3082,52248,5698,3.0,38,2.5,3.0,12.0,12.0,3.0,3.0,...,0,1,0,1,0,1,0,1,1,0


In [ ]:
aggregate_params = {}
for col in stent_df.columns:
    print(col)
    if (col in ['stent_Comprimento Dispositivo - Stent', 'stent_Diâmetro Cateter Balão Adjunto Pré', 'stent_Diâmetro Cateter Balão Adjunto Pós', 'stent_Pressão Final de Liberação (ATM)', 'stent_Grau de estenose pós', 'stent_Timi pós', 'stent_Blush Miocárdico']):
        aggregate_params[col] = ['mean', 'max', 'min']
    elif (col in ['stent_Diâmetro Dispositivo - Stent']):
        aggregate_params[col] = ['mean', 'max', 'min', 'count']
    elif (col in ['stent_N. Injeções - Adenosina - Stent', 'stent_Total Injetado (mg) - Adenosina - Stent', 'stent_N. Injeções - Papaverina - Stent', 'stent_Total Injetado (mg) - Papaverina - Stent', 'stent_N. Injeções - Nitroglicerina - Stent', 'stent_Total Injetado (mg) - Nitroglicerina - Stent', 'stent_N. Injeções - Monocordil - Stent', 'stent_Total Injetado (mg) - Monocordil - Stent', 'stent_N. Injeções - Nitroprussiato - Stent', 'stent_Total Injetado (mg) - Nitroprussiato - Stent', 'stent_N. Injeções - Adrenalina - Stent', 'stent_Total Injetado (mg) - Adrenalina - Stent']):
        aggregate_params[col] = ['sum']
    for col_encoding in cols_encoding:
        if (col_encoding in col):
            aggregate_params[col] = ['sum']

stent_Nº. Paciente
stent_Nº da Intervenção
stent_Diâmetro Dispositivo - Stent
stent_Comprimento Dispositivo - Stent
stent_Diâmetro Cateter Balão Adjunto Pré
stent_Diâmetro Cateter Balão Adjunto Pós
stent_Pressão Final de Liberação (ATM)
stent_Grau de estenose pós
stent_Timi pós
stent_Blush Miocárdico
stent_N. Injeções - Adenosina - Stent
stent_Total Injetado (mg) - Adenosina - Stent
stent_N. Injeções - Papaverina - Stent
stent_Total Injetado (mg) - Papaverina - Stent
stent_N. Injeções - Nitroglicerina - Stent
stent_Total Injetado (mg) - Nitroglicerina - Stent
stent_N. Injeções - Monocordil - Stent
stent_Total Injetado (mg) - Monocordil - Stent
stent_N. Injeções - Nitroprussiato - Stent
stent_Total Injetado (mg) - Nitroprussiato - Stent
stent_N. Injeções - Adrenalina - Stent
stent_Total Injetado (mg) - Adrenalina - Stent
stent_Vaso coronário - Stent_CD
stent_Vaso coronário - Stent_CX
stent_Vaso coronário - Stent_DA
stent_Vaso coronário - Stent_DPD
stent_Vaso coronário - Stent_DPE
stent_

In [ ]:
pp.pprint(aggregate_params)

{   'stent_Adenosina - Stent_Não': ['sum'],
    'stent_Adenosina - Stent_Sim': ['sum'],
    'stent_Adrenalina - Stent_Não': ['sum'],
    'stent_Adrenalina - Stent_Sim': ['sum'],
    'stent_Blush Miocárdico': ['mean', 'max', 'min'],
    'stent_Classificação da Lesão (ACC/AHA) - Stent_A': ['sum'],
    'stent_Classificação da Lesão (ACC/AHA) - Stent_B1': ['sum'],
    'stent_Classificação da Lesão (ACC/AHA) - Stent_B2': ['sum'],
    'stent_Classificação da Lesão (ACC/AHA) - Stent_C': ['sum'],
    'stent_Comprimento Dispositivo - Stent': ['mean', 'max', 'min'],
    'stent_Distúrbio de Fluxo - Stent_Não': ['sum'],
    'stent_Distúrbio de Fluxo - Stent_Sim': ['sum'],
    'stent_Diâmetro Cateter Balão Adjunto Pré': ['mean', 'max', 'min'],
    'stent_Diâmetro Cateter Balão Adjunto Pós': ['mean', 'max', 'min'],
    'stent_Diâmetro Dispositivo - Stent': ['mean', 'max', 'min', 'count'],
    'stent_Estratégia_Pré e Pós dilatação': ['sum'],
    'stent_Estratégia_Pré-dilatação': ['sum'],
    'stent_E

In [ ]:
stent_agg_df = stent_df.groupby(['stent_Nº. Paciente','stent_Nº da Intervenção']).agg(aggregate_params)
stent_agg_df.reset_index(inplace=True)
stent_agg_df

stent_Nº. Paciente stent_Nº da Intervenção  \
                                                  
0                  1005                   38117   
1                  1006                   38112   
2                  1016                   38193   
3                  1017                   38128   
4                  1018                   38126   
...                 ...                     ...   
9140              52140                    5625   
9141              52223                    1454   
9142              52237                   69579   
9143              52248                    5698   
9144              52370                    5662   

     stent_Diâmetro Dispositivo - Stent                  \
                                   mean  max  min count   
0                                   3.0  3.0  3.0     1   
1                                   2.5  2.5  2.5     1   
2                                   3.0  3.0  3.0     1   
3                                   3.0  3.0  3.0     1   
4                                   3.0  3.0  3.0     1   
...                                 ...  ...  ...   ...   
9140                                3.5  3.5  3.5     1   
9141                                3.0  3.0  3.0     1   
9142                                3.0  3.0  3.0     1   
9143                                3.0  3.0  3.0     1   
9144                                3.5  3.5  3.5     1   

     stent_Comprimento Dispositivo - Stent          \
                                      mean max min   
0                                      8.0   8   8   
1                                     12.0  12  12   
2                                     16.0  16  16   
3                                     16.0  16  16   
4                                     28.0  28  28   
...                                    ...  ..  ..   
9140                                  28.0  28  28   
9141                                  38.0  38  38   
9142                                  16.0  16  16   
9143                                  38.0  38  38   
9144                                  38.0  38  38   

     stent_Diâmetro Cateter Balão Adjunto Pré  ...  \
                                         mean  ...   
0                                         NaN  ...   
1                                         2.5  ...   
2                                         2.5  ...   
3                                         NaN  ...   
4                                         NaN  ...   
...                                       ...  ...   
9140                                      NaN  ...   
9141                                      2.5  ...   
9142                                      NaN  ...   
9143                                      2.5  ...   
9144                                      NaN  ...   

     stent_Adenosina - Stent_Sim stent_Papaverina - Stent_Não  \
                             sum                          sum   
0                              0                            1   
1                              0                            1   
2                              0                            1   
3                              0                            1   
4                              0                            1   
...                          ...                          ...   
9140                           1                            1   
9141                           0                            1   
9142                           0                            1   
9143                           0                            1   
9144                           0                            1   

     stent_Papaverina - Stent_Sim stent_Nitroglicerina - Stent_Não  \
                              sum                              sum   
0                               0                                1   
1                               0                                1   
2                               0        

In [ ]:
print('Index length', len(stent_agg_df.index))
print('Unique', len(stent_agg_df.index.unique()))

Index length 9145
Unique 9145


## Vaso

#### Operations
| Variável                                      | Operação                                                                     |
|-----------------------------------------------|------------------------------------------------------------------------------|
| vaso_Nº. Paciente                             | CHAVE DE CRUZAMENTO E AGREGADOR                                              |
| vaso_Nº da Intervenção                        | CHAVE DE CRUZAMENTO E AGREGADOR                                              |
| vaso_Anatomia - Intervenção                   | enconding e contagem por paciente+intervenção                                |
| vaso_Vaso coronário - Intervenção             | enconding e contagem por paciente+intervenção                                |
| vaso_Local da Lesão - Intervenção             | enconding e contagem por paciente+intervenção                                |
| vaso_Tipo de Lesão - Intervenção              | enconding e contagem por paciente+intervenção                                |
| vaso_Tipo de Lesão - Intrastent - Intervenção | enconding e contagem por paciente+intervenção                                |
| vaso_Tipo de Lesão - Focal - Intervenção      | enconding e contagem por paciente+intervenção                                |
| vaso_Tipo de Lesão - Difusa - Intervenção     | enconding e contagem por paciente+intervenção                                |
| vaso_Tipo de Lesão - Pós balão - Intervenção  | enconding e contagem por paciente+intervenção                                |
| vaso_Calcificação - Intervenção               | enconding e contagem por paciente+intervenção                                |
| vaso_Tortuosidade - Intervenção               | enconding e contagem por paciente+intervenção                                |
| vaso_Angulação - Intervenção                  | enconding e contagem por paciente+intervenção                                |
| vaso_Grau de Importância - Intervenção        | enconding e contagem por paciente+intervenção                                |
| vaso_Maior que 20mm - Intervenção             | enconding e contagem por paciente+intervenção                                |
| vaso_Trombo - Intervenção                     | enconding e contagem por paciente+intervenção                                |
| vaso_Placa Rota - Intervenção                 | enconding e contagem por paciente+intervenção                                |
| vaso_Placa Ulcerada - Intervenção             | enconding e contagem por paciente+intervenção                                |
| vaso_Ectasia - Intervenção                    | enconding e contagem por paciente+intervenção                                |
| vaso_Aneurisma - Intervenção                  | enconding e contagem por paciente+intervenção                                |
| vaso_Bifurcação - Intervenção                 | fazer o enconding mas verificar antes se estão presentes as 8 posssibiidades |
| vaso_Grau de estenose pré - Intervenção       | máx, min, media, soma                                                        |
| vaso_Convênio                                 | x                                                                            |

In [ ]:
cols_to_drop = ['vaso_Convênio']
vaso_df.drop(columns=cols_to_drop, inplace=True)

/home/titobr/mestrado/mestrado-venv/lib/python3.6/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
cols_encoding = ['vaso_Anatomia - Intervenção', 'vaso_Vaso coronário - Intervenção', \
                 'vaso_Local da Lesão - Intervenção', 'vaso_Tipo de Lesão - Intervenção', \
                 'vaso_Tipo de Lesão - Intrastent - Intervenção', 'vaso_Tipo de Lesão - Focal - Intervenção', \
                 'vaso_Tipo de Lesão - Difusa - Intervenção', 'vaso_Tipo de Lesão - Pós balão - Intervenção', \
                 'vaso_Calcificação - Intervenção', 'vaso_Tortuosidade - Intervenção', 'vaso_Angulação - Intervenção', \
                 'vaso_Grau de Importância - Intervenção', 'vaso_Maior que 20mm - Intervenção', \
                 'vaso_Trombo - Intervenção', 'vaso_Placa Rota - Intervenção', 'vaso_Placa Ulcerada - Intervenção', \
                 'vaso_Ectasia - Intervenção', 'vaso_Aneurisma - Intervenção', 'vaso_Bifurcação - Intervenção']
vaso_df = pd.get_dummies(vaso_df, columns = cols_encoding)

In [ ]:
vaso_df.sort_values(by=['vaso_Nº. Paciente'])

,vaso_Nº. Paciente,vaso_Nº da Intervenção,vaso_Grau de estenose pré - Intervenção,vaso_Anatomia - Intervenção_Ateromatose moderada,vaso_Anatomia - Intervenção_Ateromatose severa,vaso_Anatomia - Intervenção_Com irregularidades parietais,vaso_Anatomia - Intervenção_Com lesão,vaso_Vaso coronário - Intervenção_CD,vaso_Vaso coronário - Intervenção_CX,vaso_Vaso coronário - Intervenção_DA,...,vaso_Ectasia - Intervenção_Sim,vaso_Aneurisma - Intervenção_Não,vaso_Aneurisma - Intervenção_Sim,"vaso_Bifurcação - Intervenção_0,0,1","vaso_Bifurcação - Intervenção_0,1,0","vaso_Bifurcação - Intervenção_0,1,1","vaso_Bifurcação - Intervenção_1,0,0","vaso_Bifurcação - Intervenção_1,0,1","vaso_Bifurcação - Intervenção_1,1,0","vaso_Bifurcação - Intervenção_1,1,1"
3352,1000,38108,99.0,0,0,0,1,0,0,1,...,0,1,0,0,0,0,0,0,0,0
8218,1005,38117,70.0,0,0,0,1,1,0,0,...,0,1,0,0,0,0,0,0,0,0
9767,1006,38112,80.0,0,0,0,1,0,1,0,...,0,1,0,0,0,0,0,0,0,0
4,1016,38193,95.0,0,0,0,1,1,0,0,...,0,1,0,0,0,0,0,0,0,0
5,1017,38128,70.0,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9765,52223,1454,100.0,0,0,0,1,0,0,1,...,0,1,0,0,0,0,0,0,0,0
3350,52223,1454,100.0,0,0,0,1,1,0,0,...,0,1,0,0,0,0,0,0,0,0
1709,52237,69579,90.0,0,0,0,1,0,0,1,...,0,1,0,0,0,0,0,0,0,0
3351,52248,5698,70.0,0,0,0,1,0,0,1,...,0,1,0,0,0,0,0,0,0,0


In [ ]:
aggregate_params = {}
for col in vaso_df.columns:
    print(col)
    if (col in ['vaso_Grau de estenose pré - Intervenção']):
        aggregate_params[col] = ['max', 'min', 'mean', 'sum']
    for col_encoding in cols_encoding:
        if (col_encoding in col):
            aggregate_params[col] = ['sum']

vaso_Nº. Paciente
vaso_Nº da Intervenção
vaso_Grau de estenose pré - Intervenção
vaso_Anatomia - Intervenção_Ateromatose moderada
vaso_Anatomia - Intervenção_Ateromatose severa
vaso_Anatomia - Intervenção_Com irregularidades parietais
vaso_Anatomia - Intervenção_Com lesão
vaso_Vaso coronário - Intervenção_CD
vaso_Vaso coronário - Intervenção_CX
vaso_Vaso coronário - Intervenção_DA
vaso_Vaso coronário - Intervenção_DPD
vaso_Vaso coronário - Intervenção_DPE
vaso_Vaso coronário - Intervenção_Dg1
vaso_Vaso coronário - Intervenção_Dg2
vaso_Vaso coronário - Intervenção_Dg3
vaso_Vaso coronário - Intervenção_Intermédio-RI
vaso_Vaso coronário - Intervenção_MamáriaCD
vaso_Vaso coronário - Intervenção_MamáriaDA
vaso_Vaso coronário - Intervenção_MamáriaDg
vaso_Vaso coronário - Intervenção_MamáriaMg
vaso_Vaso coronário - Intervenção_Mg1
vaso_Vaso coronário - Intervenção_Mg2
vaso_Vaso coronário - Intervenção_Mg3
vaso_Vaso coronário - Intervenção_Mg4
vaso_Vaso coronário - Intervenção_MgD
vaso_Vaso co

In [ ]:
pp.pprint(aggregate_params)

{   'vaso_Anatomia - Intervenção_Ateromatose moderada': ['sum'],
    'vaso_Anatomia - Intervenção_Ateromatose severa': ['sum'],
    'vaso_Anatomia - Intervenção_Com irregularidades parietais': ['sum'],
    'vaso_Anatomia - Intervenção_Com lesão': ['sum'],
    'vaso_Aneurisma - Intervenção_Não': ['sum'],
    'vaso_Aneurisma - Intervenção_Sim': ['sum'],
    'vaso_Angulação - Intervenção_>45': ['sum'],
    'vaso_Angulação - Intervenção_>90': ['sum'],
    'vaso_Angulação - Intervenção_Acentuada': ['sum'],
    'vaso_Angulação - Intervenção_Leve': ['sum'],
    'vaso_Angulação - Intervenção_Moderada': ['sum'],
    'vaso_Bifurcação - Intervenção_0,0,1': ['sum'],
    'vaso_Bifurcação - Intervenção_0,1,0': ['sum'],
    'vaso_Bifurcação - Intervenção_0,1,1': ['sum'],
    'vaso_Bifurcação - Intervenção_1,0,0': ['sum'],
    'vaso_Bifurcação - Intervenção_1,0,1': ['sum'],
    'vaso_Bifurcação - Intervenção_1,1,0': ['sum'],
    'vaso_Bifurcação - Intervenção_1,1,1': ['sum'],
    'vaso_Calcificação - 

In [ ]:
vaso_agg_df = vaso_df.groupby(['vaso_Nº. Paciente','vaso_Nº da Intervenção']).agg(aggregate_params)
vaso_agg_df.reset_index(inplace=True)
vaso_agg_df

vaso_Nº. Paciente vaso_Nº da Intervenção  \
                                                
0                 1000                  38108   
1                 1005                  38117   
2                 1006                  38112   
3                 1016                  38193   
4                 1017                  38128   
...                ...                    ...   
9629             52140                   5625   
9630             52223                   1454   
9631             52237                  69579   
9632             52248                   5698   
9633             52370                   5662   

     vaso_Grau de estenose pré - Intervenção                       \
                                         max    min   mean    sum   
0                                       99.0   99.0   99.0   99.0   
1                                       70.0   70.0   70.0   70.0   
2                                       80.0   80.0   80.0   80.0   
3                                       95.0   95.0   95.0   95.0   
4                                       70.0   70.0   70.0  140.0   
...                                      ...    ...    ...    ...   
9629                                   100.0  100.0  100.0  100.0   
9630                                   100.0  100.0  100.0  200.0   
9631                                    90.0   90.0   90.0   90.0   
9632                                    70.0   70.0   70.0   70.0   
9633                                    90.0   90.0   90.0   90.0   

     vaso_Anatomia - Intervenção_Ateromatose moderada  \
                                                  sum   
0                                                   0   
1                                                   0   
2                                                   0   
3                                                   0   
4                                                   0   
...                                               ...   
9629                                                0   
9630                                                0   
9631                                                0   
9632                                                0   
9633                                                0   

     vaso_Anatomia - Intervenção_Ateromatose severa  \
                                                sum   
0                                                 0   
1                                                 0   
2                                                 0   
3                                                 0   
4                                                 0   
...                                             ...   
9629                                              0   
9630                                              0   
9631                                              0   
9632                                              0   
9633                                              0   

     vaso_Anatomia - Intervenção_Com irregularidades parietais  \
                                                           sum   
0                                                     0          
1                                                     0          
2                                                     0          
3                                                     0          
4                                                     0          
...                                                 ...          
9629                                                  0          
9630                                                  0          
9631                                                  0          
9632                                                  0          
9633                                                  0          

     vaso_Anatomia - Intervenção_Com lesão  ...  \
                                       sum  ...   
0                                        1  ...   

In [ ]:
print('Index length', len(vaso_agg_df.index))
print('Unique', len(vaso_agg_df.index.unique()))

Index length 9634
Unique 9634


# Merging

In [ ]:
def rename_columns(df):
    rename = {}
    for col in df.columns:
        new_col_name = col
        #rename tuples
        if len(col)==2:
            if col[1]=='':
                new_col_name = col[0]
            else:
                new_col_name = col[0]+'_'+col[1]
        #rename replacing chars [,]<>
        new_col_name = new_col_name.replace('[','').replace(']','').replace(',','').replace('<','lt').replace('>','gt')
        print('New col name', new_col_name)
        rename[col]=new_col_name
    return rename

In [ ]:
merge_df = pd.merge(evolucao_df, paciente_df,  how='left', left_on=['evolucao_Nº. Paciente'], right_on = ['paciente_Nº. Paciente'])
merge_df.rename(columns=rename_columns(merge_df), inplace=True)
merge_df

New col name evolucao_Nº. Paciente
New col name evolucao_Nº da Intervenção
New col name evolucao_Problema Clínico
New col name evolucao_Infarto após Intervenção
New col name evolucao_Óbito Hospitalar
New col name evolucao_Complicação Vascular
New col name evolucao_I.R.A
New col name evolucao_I.R.A Dialítica
New col name evolucao_A.V.C Hemorrágico
New col name evolucao_A.V.C Isquêmico
New col name paciente_Nº. Paciente
New col name paciente_Idade Atual
New col name paciente_Idade no procedimento
New col name paciente_Frequência Cardíaca
New col name paciente_Frequência Respiratória
New col name paciente_Temperatura (ºC)
New col name paciente_Peso (Kg)
New col name paciente_Altura (metros)
New col name paciente_Índice de Massa Corporal
New col name paciente_Creatinina Sérica
New col name paciente_MDRD (ml/min/173m²)
New col name paciente_Clearance Cr (ml/min)
New col name paciente_Pressão Arterial_min
New col name paciente_Pressão Arterial_max
New col name paciente_Pressão Arterial Diast

,evolucao_Nº. Paciente,evolucao_Nº da Intervenção,evolucao_Problema Clínico,evolucao_Infarto após Intervenção,evolucao_Óbito Hospitalar,evolucao_Complicação Vascular,evolucao_I.R.A,evolucao_I.R.A Dialítica,evolucao_A.V.C Hemorrágico,evolucao_A.V.C Isquêmico,...,paciente_Alergia ao iodo_Não,paciente_Alergia ao iodo_Sim,paciente_DAC conhecida_Não,paciente_DAC conhecida_Sim,paciente_Cate Prévio_Não,paciente_Cate Prévio_Sim,paciente_Cirurgia de Revascularização Prévia_Não,paciente_Cirurgia de Revascularização Prévia_Sim,paciente_Intervenção Percutânea Prévia_Não,paciente_Intervenção Percutânea Prévia_Sim
0,5700,42353,0,0,0,0,0,0,0,0,...,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
1,8300,46156,0,0,0,0,0,0,0,0,...,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0
2,8500,45437,0,0,0,0,0,0,0,0,...,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0
3,1057,38316,0,0,0,0,0,0,0,0,...,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
4,1061,57118,0,0,0,0,0,0,0,0,...,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9631,50643,5421,0,0,0,0,0,0,0,0,...,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
9632,51552,5548,1,0,1,0,0,0,0,0,...,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
9633,51585,69452,0,0,0,0,0,0,0,0,...,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0
9634,52140,5625,0,0,0,0,0,0,0,0,...,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0


In [ ]:
merge_df = pd.merge(merge_df, procedimento_agg_df,  how='left', left_on=['evolucao_Nº. Paciente','evolucao_Nº da Intervenção'], right_on = ['procedimento_Nº. Paciente','procedimento_Nº da Intervenção'])
merge_df.rename(columns=rename_columns(merge_df), inplace=True)
merge_df

New col name evolucao_Nº. Paciente
New col name evolucao_Nº da Intervenção
New col name evolucao_Problema Clínico
New col name evolucao_Infarto após Intervenção
New col name evolucao_Óbito Hospitalar
New col name evolucao_Complicação Vascular
New col name evolucao_I.R.A
New col name evolucao_I.R.A Dialítica
New col name evolucao_A.V.C Hemorrágico
New col name evolucao_A.V.C Isquêmico
New col name paciente_Nº. Paciente
New col name paciente_Idade Atual
New col name paciente_Idade no procedimento
New col name paciente_Frequência Cardíaca
New col name paciente_Frequência Respiratória
New col name paciente_Temperatura (ºC)
New col name paciente_Peso (Kg)
New col name paciente_Altura (metros)
New col name paciente_Índice de Massa Corporal
New col name paciente_Creatinina Sérica
New col name paciente_MDRD (ml/min/173m²)
New col name paciente_Clearance Cr (ml/min)
New col name paciente_Pressão Arterial_min
New col name paciente_Pressão Arterial_max
New col name paciente_Pressão Arterial Diast

/home/titobr/mestrado/mestrado-venv/lib/python3.6/site-packages/pandas/core/reshape/merge.py:618: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)


,evolucao_Nº. Paciente,evolucao_Nº da Intervenção,evolucao_Problema Clínico,evolucao_Infarto após Intervenção,evolucao_Óbito Hospitalar,evolucao_Complicação Vascular,evolucao_I.R.A,evolucao_I.R.A Dialítica,evolucao_A.V.C Hemorrágico,evolucao_A.V.C Isquêmico,...,procedimento_Ultrasom Intracoronário_Não_sum,procedimento_Ultrasom Intracoronário_Sim_sum,procedimento_Angiografia Quantitativa_Não_sum,procedimento_Angiografia Quantitativa_Sim_sum,procedimento_Inibidor de glicoproteína_Agrastat_sum,procedimento_Inibidor de glicoproteína_Nenhum_sum,procedimento_Inibidor de glicoproteína_Reo-Pro_sum,procedimento_Clopidogrel.1_lt 6h_sum,procedimento_Clopidogrel.1_gt 6h_sum,procedimento_Clopidogrel.1_Não_sum
0,5700,42353,0,0,0,0,0,0,0,0,...,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
1,8300,46156,0,0,0,0,0,0,0,0,...,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
2,8500,45437,0,0,0,0,0,0,0,0,...,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
3,1057,38316,0,0,0,0,0,0,0,0,...,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
4,1061,57118,0,0,0,0,0,0,0,0,...,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9631,50643,5421,0,0,0,0,0,0,0,0,...,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
9632,51552,5548,1,0,1,0,0,0,0,0,...,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
9633,51585,69452,0,0,0,0,0,0,0,0,...,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
9634,52140,5625,0,0,0,0,0,0,0,0,...,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0


In [ ]:
merge_df = pd.merge(merge_df, complicacao_agg_df,  how='left', left_on=['evolucao_Nº. Paciente','evolucao_Nº da Intervenção'], right_on = ['complicacao_Nº. Paciente','complicacao_Nº da Intervenção'])
merge_df.rename(columns=rename_columns(merge_df), inplace=True)
merge_df

New col name evolucao_Nº. Paciente
New col name evolucao_Nº da Intervenção
New col name evolucao_Problema Clínico
New col name evolucao_Infarto após Intervenção
New col name evolucao_Óbito Hospitalar
New col name evolucao_Complicação Vascular
New col name evolucao_I.R.A
New col name evolucao_I.R.A Dialítica
New col name evolucao_A.V.C Hemorrágico
New col name evolucao_A.V.C Isquêmico
New col name paciente_Nº. Paciente
New col name paciente_Idade Atual
New col name paciente_Idade no procedimento
New col name paciente_Frequência Cardíaca
New col name paciente_Frequência Respiratória
New col name paciente_Temperatura (ºC)
New col name paciente_Peso (Kg)
New col name paciente_Altura (metros)
New col name paciente_Índice de Massa Corporal
New col name paciente_Creatinina Sérica
New col name paciente_MDRD (ml/min/173m²)
New col name paciente_Clearance Cr (ml/min)
New col name paciente_Pressão Arterial_min
New col name paciente_Pressão Arterial_max
New col name paciente_Pressão Arterial Diast

/home/titobr/mestrado/mestrado-venv/lib/python3.6/site-packages/pandas/core/reshape/merge.py:618: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)


,evolucao_Nº. Paciente,evolucao_Nº da Intervenção,evolucao_Problema Clínico,evolucao_Infarto após Intervenção,evolucao_Óbito Hospitalar,evolucao_Complicação Vascular,evolucao_I.R.A,evolucao_I.R.A Dialítica,evolucao_A.V.C Hemorrágico,evolucao_A.V.C Isquêmico,...,complicacao_Tipo Complicação_Isquêmica_sum,complicacao_Tipo Complicação_Neurológica_sum,complicacao_Tipo Complicação_R. Contraste_sum,complicacao_Tipo Complicação_Vagal_sum,complicacao_Tipo Complicação_Vascular_sum,complicacao_Grau - Complicação_Grave_sum,complicacao_Grau - Complicação_Leve_sum,complicacao_Grau - Complicação_Moderado_sum,complicacao_Conduta_Orientação Telefônica_sum,complicacao_Conduta_Retorno para avaliação Médica_sum
0,5700,42353,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,8300,46156,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,8500,45437,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1057,38316,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,1061,57118,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9631,50643,5421,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
9632,51552,5548,1,0,1,0,0,0,0,0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
9633,51585,69452,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9634,52140,5625,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
merge_df = pd.merge(merge_df, balao_agg_df,  how='left', left_on=['evolucao_Nº. Paciente','evolucao_Nº da Intervenção'], right_on = ['balao_Nº. Paciente','balao_Nº da Intervenção'])
merge_df.rename(columns=rename_columns(merge_df), inplace=True)
merge_df

/home/titobr/mestrado/mestrado-venv/lib/python3.6/site-packages/pandas/core/reshape/merge.py:618: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)


New col name evolucao_Nº. Paciente
New col name evolucao_Nº da Intervenção
New col name evolucao_Problema Clínico
New col name evolucao_Infarto após Intervenção
New col name evolucao_Óbito Hospitalar
New col name evolucao_Complicação Vascular
New col name evolucao_I.R.A
New col name evolucao_I.R.A Dialítica
New col name evolucao_A.V.C Hemorrágico
New col name evolucao_A.V.C Isquêmico
New col name paciente_Nº. Paciente
New col name paciente_Idade Atual
New col name paciente_Idade no procedimento
New col name paciente_Frequência Cardíaca
New col name paciente_Frequência Respiratória
New col name paciente_Temperatura (ºC)
New col name paciente_Peso (Kg)
New col name paciente_Altura (metros)
New col name paciente_Índice de Massa Corporal
New col name paciente_Creatinina Sérica
New col name paciente_MDRD (ml/min/173m²)
New col name paciente_Clearance Cr (ml/min)
New col name paciente_Pressão Arterial_min
New col name paciente_Pressão Arterial_max
New col name paciente_Pressão Arterial Diast

,evolucao_Nº. Paciente,evolucao_Nº da Intervenção,evolucao_Problema Clínico,evolucao_Infarto após Intervenção,evolucao_Óbito Hospitalar,evolucao_Complicação Vascular,evolucao_I.R.A,evolucao_I.R.A Dialítica,evolucao_A.V.C Hemorrágico,evolucao_A.V.C Isquêmico,...,balao_Vaso coronário - Balão_PVSMg1_sum,balao_Vaso coronário - Balão_PVSMg2_sum,balao_Vaso coronário - Balão_RadialDg_sum,balao_Vaso coronário - Balão_TCE_sum,balao_Vaso coronário - Balão_VPD_sum,balao_Vaso coronário - Balão_VPE_sum,balao_Kissing wire - Balão_Não_sum,balao_Kissing wire - Balão_Sim_sum,balao_Kissing balloon - Balão_Não_sum,balao_Kissing balloon - Balão_Sim_sum
0,5700,42353,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,8300,46156,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,8500,45437,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1057,38316,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1061,57118,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9631,50643,5421,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9632,51552,5548,1,0,1,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9633,51585,69452,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9634,52140,5625,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
merge_df = pd.merge(merge_df, stent_agg_df,  how='left', left_on=['evolucao_Nº. Paciente','evolucao_Nº da Intervenção'], right_on = ['stent_Nº. Paciente','stent_Nº da Intervenção'])
merge_df.rename(columns=rename_columns(merge_df), inplace=True)
merge_df

/home/titobr/mestrado/mestrado-venv/lib/python3.6/site-packages/pandas/core/reshape/merge.py:618: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)


New col name evolucao_Nº. Paciente
New col name evolucao_Nº da Intervenção
New col name evolucao_Problema Clínico
New col name evolucao_Infarto após Intervenção
New col name evolucao_Óbito Hospitalar
New col name evolucao_Complicação Vascular
New col name evolucao_I.R.A
New col name evolucao_I.R.A Dialítica
New col name evolucao_A.V.C Hemorrágico
New col name evolucao_A.V.C Isquêmico
New col name paciente_Nº. Paciente
New col name paciente_Idade Atual
New col name paciente_Idade no procedimento
New col name paciente_Frequência Cardíaca
New col name paciente_Frequência Respiratória
New col name paciente_Temperatura (ºC)
New col name paciente_Peso (Kg)
New col name paciente_Altura (metros)
New col name paciente_Índice de Massa Corporal
New col name paciente_Creatinina Sérica
New col name paciente_MDRD (ml/min/173m²)
New col name paciente_Clearance Cr (ml/min)
New col name paciente_Pressão Arterial_min
New col name paciente_Pressão Arterial_max
New col name paciente_Pressão Arterial Diast

,evolucao_Nº. Paciente,evolucao_Nº da Intervenção,evolucao_Problema Clínico,evolucao_Infarto após Intervenção,evolucao_Óbito Hospitalar,evolucao_Complicação Vascular,evolucao_I.R.A,evolucao_I.R.A Dialítica,evolucao_A.V.C Hemorrágico,evolucao_A.V.C Isquêmico,...,stent_Adenosina - Stent_Sim_sum,stent_Papaverina - Stent_Não_sum,stent_Papaverina - Stent_Sim_sum,stent_Nitroglicerina - Stent_Não_sum,stent_Nitroglicerina - Stent_Sim_sum,stent_Monocordil - Stent_Não_sum,stent_Monocordil - Stent_Sim_sum,stent_Nitroprussiato - Stent_Não_sum,stent_Adrenalina - Stent_Não_sum,stent_Adrenalina - Stent_Sim_sum
0,5700,42353,0,0,0,0,0,0,0,0,...,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0
1,8300,46156,0,0,0,0,0,0,0,0,...,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0
2,8500,45437,0,0,0,0,0,0,0,0,...,0.0,2.0,0.0,2.0,0.0,2.0,0.0,2.0,2.0,0.0
3,1057,38316,0,0,0,0,0,0,0,0,...,0.0,2.0,0.0,2.0,0.0,2.0,0.0,2.0,2.0,0.0
4,1061,57118,0,0,0,0,0,0,0,0,...,0.0,2.0,0.0,2.0,0.0,2.0,0.0,2.0,2.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9631,50643,5421,0,0,0,0,0,0,0,0,...,0.0,3.0,0.0,3.0,0.0,3.0,0.0,3.0,3.0,0.0
9632,51552,5548,1,0,1,0,0,0,0,0,...,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0
9633,51585,69452,0,0,0,0,0,0,0,0,...,0.0,2.0,0.0,2.0,0.0,2.0,0.0,2.0,2.0,0.0
9634,52140,5625,0,0,0,0,0,0,0,0,...,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0


In [ ]:
merge_df = pd.merge(merge_df, vaso_agg_df,  how='left', left_on=['evolucao_Nº. Paciente','evolucao_Nº da Intervenção'], right_on = ['vaso_Nº. Paciente','vaso_Nº da Intervenção'])
merge_df.rename(columns=rename_columns(merge_df), inplace=True)
merge_df

New col name evolucao_Nº. Paciente
New col name evolucao_Nº da Intervenção
New col name evolucao_Problema Clínico
New col name evolucao_Infarto após Intervenção
New col name evolucao_Óbito Hospitalar
New col name evolucao_Complicação Vascular
New col name evolucao_I.R.A
New col name evolucao_I.R.A Dialítica
New col name evolucao_A.V.C Hemorrágico
New col name evolucao_A.V.C Isquêmico
New col name paciente_Nº. Paciente
New col name paciente_Idade Atual
New col name paciente_Idade no procedimento
New col name paciente_Frequência Cardíaca
New col name paciente_Frequência Respiratória
New col name paciente_Temperatura (ºC)
New col name paciente_Peso (Kg)
New col name paciente_Altura (metros)
New col name paciente_Índice de Massa Corporal
New col name paciente_Creatinina Sérica
New col name paciente_MDRD (ml/min/173m²)
New col name paciente_Clearance Cr (ml/min)
New col name paciente_Pressão Arterial_min
New col name paciente_Pressão Arterial_max
New col name paciente_Pressão Arterial Diast

/home/titobr/mestrado/mestrado-venv/lib/python3.6/site-packages/pandas/core/reshape/merge.py:618: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)


,evolucao_Nº. Paciente,evolucao_Nº da Intervenção,evolucao_Problema Clínico,evolucao_Infarto após Intervenção,evolucao_Óbito Hospitalar,evolucao_Complicação Vascular,evolucao_I.R.A,evolucao_I.R.A Dialítica,evolucao_A.V.C Hemorrágico,evolucao_A.V.C Isquêmico,...,vaso_Ectasia - Intervenção_Sim_sum,vaso_Aneurisma - Intervenção_Não_sum,vaso_Aneurisma - Intervenção_Sim_sum,vaso_Bifurcação - Intervenção_001_sum,vaso_Bifurcação - Intervenção_010_sum,vaso_Bifurcação - Intervenção_011_sum,vaso_Bifurcação - Intervenção_100_sum,vaso_Bifurcação - Intervenção_101_sum,vaso_Bifurcação - Intervenção_110_sum,vaso_Bifurcação - Intervenção_111_sum
0,5700,42353,0,0,0,0,0,0,0,0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,8300,46156,0,0,0,0,0,0,0,0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,8500,45437,0,0,0,0,0,0,0,0,...,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1057,38316,0,0,0,0,0,0,0,0,...,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1061,57118,0,0,0,0,0,0,0,0,...,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9631,50643,5421,0,0,0,0,0,0,0,0,...,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
9632,51552,5548,1,0,1,0,0,0,0,0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
9633,51585,69452,0,0,0,0,0,0,0,0,...,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9634,52140,5625,0,0,0,0,0,0,0,0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
merge_df.to_csv('V2_merge_df.csv')

## Split Train Test

In [ ]:
paciente_testDataset_indexes = pd.read_csv('paciente_testDataset_indices.csv', index_col=0, header=None)
print(len(paciente_testDataset_indexes.index.unique()))
paciente_testDataset_indexes.head()

1750


""
0
28456.0
18805.0
42150.0
9296.0
21273.0


In [ ]:
test_df  = merge_df[merge_df['evolucao_Nº. Paciente'].isin(paciente_testDataset_indexes.index)]
print(len(test_df['evolucao_Nº. Paciente'].unique()))
test_df.head(1)

1750


,evolucao_Nº. Paciente,evolucao_Nº da Intervenção,evolucao_Problema Clínico,evolucao_Infarto após Intervenção,evolucao_Óbito Hospitalar,evolucao_Complicação Vascular,evolucao_I.R.A,evolucao_I.R.A Dialítica,evolucao_A.V.C Hemorrágico,evolucao_A.V.C Isquêmico,...,vaso_Ectasia - Intervenção_Sim_sum,vaso_Aneurisma - Intervenção_Não_sum,vaso_Aneurisma - Intervenção_Sim_sum,vaso_Bifurcação - Intervenção_001_sum,vaso_Bifurcação - Intervenção_010_sum,vaso_Bifurcação - Intervenção_011_sum,vaso_Bifurcação - Intervenção_100_sum,vaso_Bifurcação - Intervenção_101_sum,vaso_Bifurcação - Intervenção_110_sum,vaso_Bifurcação - Intervenção_111_sum
4,1061,57118,0,0,0,0,0,0,0,0,...,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
train_df  = merge_df[~merge_df['evolucao_Nº. Paciente'].isin(paciente_testDataset_indexes.index)]
print(len(train_df['evolucao_Nº. Paciente'].unique()))
train_df.head(1)

6998


,evolucao_Nº. Paciente,evolucao_Nº da Intervenção,evolucao_Problema Clínico,evolucao_Infarto após Intervenção,evolucao_Óbito Hospitalar,evolucao_Complicação Vascular,evolucao_I.R.A,evolucao_I.R.A Dialítica,evolucao_A.V.C Hemorrágico,evolucao_A.V.C Isquêmico,...,vaso_Ectasia - Intervenção_Sim_sum,vaso_Aneurisma - Intervenção_Não_sum,vaso_Aneurisma - Intervenção_Sim_sum,vaso_Bifurcação - Intervenção_001_sum,vaso_Bifurcação - Intervenção_010_sum,vaso_Bifurcação - Intervenção_011_sum,vaso_Bifurcação - Intervenção_100_sum,vaso_Bifurcação - Intervenção_101_sum,vaso_Bifurcação - Intervenção_110_sum,vaso_Bifurcação - Intervenção_111_sum
0,5700,42353,0,0,0,0,0,0,0,0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
train_df.to_csv('V2_train_df.csv')
test_df.to_csv('V2_test_df.csv')